In [1]:
# import wandb
# wandb.login()

# %env WANDB_PROJECT=evaluate_LM_with_rationalization

In [2]:
# import gpt3
import logging
import math
import os

from typing import List, Dict, Any, NewType

InputDataClass = NewType("InputDataClass", Any)
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
from transformers import (
    T5Config,
    T5ForConditionalGeneration,
    T5Tokenizer,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback
)
from transformers.trainer_utils import EvaluationStrategy
from transformers.integrations import TensorBoardCallback
import transformers
from transformers import Trainer

#from feature_conversion_methods import format_instance

from custom_args import (
    DataTrainingArguments,
    ModelArguments
)
from metrics import evaluate
import torch
import datasets
import git
import time
from datetime import datetime
import sys
from tqdm import trange
import random 
import pandas as pd 
import jsonlines
from copy import deepcopy 

logger = logging.getLogger(__name__)
transformers.logging.set_verbosity_info()
import re
def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        - level: desired level. e.g. logging.INFO. Optional. Default is logging.ERROR
        - prefices: list of one or more str prefices to match (e.g. ["transformers", "torch"]). Optional.
          Default is `[""]` to match all active loggers.
          The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)
set_global_logging_level(logging.ERROR, ["datasets"])


CONFIG_MAPPING = {"t5": T5Config}
MODEL_MAPPING = {"t5": T5ForConditionalGeneration}
TOKENIZER_MAPPING = {"t5": T5Tokenizer}


def set_other_seeds(seed):
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)

# inspired by DefaultDataCollator from:
# https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py
# modified to perform batch-level padding.
class SequenceCollator:
    def __init__(self, model, pad_token):
        self.model = model
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }

        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch


In [3]:
from collections import defaultdict
import random
"""
Example-to-Feature conversion methods
Modified from
https://github.com/salesforce/cos-e/blob/master/code/generation/train_commonsenseqa_v1.0.py and ""_v1.11.py (identical)
as well as Tensorflow code for WTF?: 
https://github.com/google-research/google-research/blob/master/wt5/wt5/preprocessors.py
"""
# This code is based on https://github.com/allenai/label_rationale_association/blob/main/feature_conversion_methods.py

unified_qa_esnli_label_mapping = {0: 'yes', 1: 'maybe', 2: 'no'}
unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'} 
wt5_esnli_label_mapping = {0: 'entailment', 1: 'neutral', 2: 'contradiction'} 
unified_qa_sbic_label_mapping = {"offensive": 'Yes', "not offensive": 'No'}

def format_instance(
        example,
        tokenizer,
        explanation_sep,
        max_seq_length=None,
        datasource=None,
        io_format=None, 
):
    assert datasource in {"cos_e", "esnli", "sbic", "sensemaking", "ecqa"}

    if datasource in ["cos_e", "ecqa"]:
        input_string, answer_string = cqa_formatting(example, io_format, explanation_sep, datasource)
    elif datasource == "esnli":
        input_string, answer_string = esnli_formatting(example, io_format, explanation_sep)
    elif datasource == 'sbic':
        input_string, answer_string = sbic_formatting(example, io_format, explanation_sep)
    elif datasource == 'sensemaking':
        input_string, answer_string = sensemaking_formatting(example, io_format, explanation_sep)
    else:
        raise ValueError("Unknown task. Currently supported: esnli, cos_e, sbic, sensemaking, ecqa.")
    
    if 'unified' in io_format and 'unifew' not in io_format:
        input_string += '</s>'

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())

    encodings = tokenizer.encode_plus(
        input_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )


    # note even with "lm_labels.shift_right()", the decoder attention mask length is still correct since we remove the last token
    dec = tokenizer.encode_plus(
        answer_string,
        max_length=max_seq_length,
        pad_to_max_length=False,
        return_token_type_ids=False,
        return_attention_mask=True,
    )

    encodings["labels"] = dec["input_ids"]
    encodings["decoder_attention_mask"] = dec["attention_mask"]
    encodings["question_encoding"] = encodings["input_ids"]

    #return encodings
    return {**example, **encodings}

#这里很简单 定义好输入输出string就可以的
def cqa_formatting(item, io_format, explanation_sep, datasource):
    question = item["question"]
    answer = item["answer"]
    abstr_expl = item["abstractive_explanation"].lower() if datasource == 'cos_e' else item["explanation"].lower()


    if io_format == 't5_fewshot_infilling_with_choices':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" The answer is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain {datasource} question: {question} context: " + ', '.join(item['choices']) # explain cos_e question: When getting in shape you need to have this in between workouts? context: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == "record": 
        # might not work because cos_e doesn't have a passage 
        input_string = f"explain {datasource} query: {question} entities: " + ', '.join(item['choices']) # explain cos_e query: When getting in shape you need to have this in between workouts? entities: give up, period of recovery, jogging
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # period of recovery because without a period of recovery you will not get any gains.
    elif io_format == 'unifiedqa_matching':
        choice_ids = ['(A)', '(B)', '(C)', '(D)', '(E)']
        input_string = f'explain {question.lower()} \\n'
        for choice_id, choice in zip(choice_ids, item["choices"]):
            input_string += f' {choice_id} {choice.lower()}'
        answer_string = f"{answer.lower()} {explanation_sep} {abstr_expl.lower()}"
        answer_string = answer_string.lower()
    elif io_format == 't5_fewshot_infilling_without_choices_use_refined_expl':
        input_string = f"explain {datasource} question: {question} choice: " + " choice: ".join(item["choices"]) + f" <extra_id_0> {explanation_sep} <extra_id_1>"
        input_string = f"explain {datasource} question: {question} answer: {answer}" + f" {explanation_sep} <extra_id_0>"
        answer_string = f"<extra_id_0> {item['our_explanation']} <extra_id_1>"

    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")
    
    return input_string, answer_string


def esnli_formatting(item, io_format, explanation_sep):

    premise = item["premise"]
    hypothesis = item["hypothesis"]
    answer = unified_qa_esnli_label_mapping[item["label"]] if 'unified' in io_format else wt5_esnli_label_mapping[item["label"]]
    abstr_expl = item["explanation_1"].lower() 
    # Dev/test instances have more than one explanation annotated; merge them into one sequence separated by [SEP] 
    for k in [2,3]:
        if f"explanation_{k}" in item and item[f'explanation_{k}']!='': 
            abstr_expl += f" [SEP] {item[f'explanation_{k}'].lower()}"

    if io_format == 'standard':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain nli hypothesis: {hypothesis} premise: {premise} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == "squad": 
        input_string = f"explain nli question: Is this entailment? context: {hypothesis} {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}" 
    elif io_format == "squad_endswith_what":
        input_string = f"explain nli question: What is this? context: {hypothesis} {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix": 
        input_string = f"explain nli question: Is this entailment? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_ynm = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer_ynm} {explanation_sep} {abstr_expl}"  
    elif io_format == "squad_nli_mix_endswith_what":  
        input_string = f"explain nli question: What is this? context: hypothesis: {hypothesis} premise: {premise}"  
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"   
    elif io_format == 'unifiedqa_unifew':
        hypothesis = hypothesis.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        answer = unified_qa_esnli_label_mapping_upper[item["label"]]
        input_string = f'explain {premise} Is {hypothesis}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_unifew_nli_mix':
        premise = premise.lower().rstrip('.')
        unified_qa_esnli_label_mapping_upper = {0: 'Yes', 1: 'Maybe', 2: 'No'}
        input_string = f'explain hypothesis: {hypothesis} Is premise: {premise}? \\n (A) Yes (B) Maybe (C) No'
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 'unifiedqa_ynm': 
        input_string = f'explain is this entailment? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm': 
        input_string = f'explain is this entailment? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}' 
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_ynm_with_choices': 
        input_string = f'explain is this entailment? \\n (A) yes (B) maybe (C) no \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = unified_qa_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_what_v2': 
        input_string = f'explain what is this? \\n {hypothesis.lower()} {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_v2': 
        input_string = f'explain what is this? \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_snli_mix_what_with_choices_v2': 
        input_string = f'explain what is this? \\n (A) entailment (B) neutral (C) contradiction \\n hypothesis: {hypothesis.lower()} premise: {premise.lower()}'  
        answer = wt5_esnli_label_mapping[item["label"]]
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 't5_fewshot_infilling_without_choices_use_refined_expl':
        answer = wt5_esnli_label_mapping[item["label"]]
        input_string = f'explain why the relation is {answer} between hypothesis: {hypothesis.lower()} and premise: {premise.lower()}'  
        #input_string = f"explain {datasource} question: {question} answer: {answer}" + f" {explanation_sep} <extra_id_0>"
        answer_string = f"<extra_id_0> {item['our_explanation']} <extra_id_1>"
    else:
        raise ValueError("The IO format is not supported.")

    return input_string, answer_string


def sbic_formatting(item, io_format, explanation_sep):
    # We pre-processed the SBIC dataset such that we join multiple implied statements with the [SEP] token for dev/test instances 
    # Each annotation in the training split is a separate instance
    post = item['post']
    answer = unified_qa_sbic_label_mapping[item["offensiveYN"]] if 'unified' in io_format else item["offensiveYN"].replace("not offensive", "not_offensive")
    abstr_expl = item["targetStereotype"]
    

    if io_format == 'cola':
        input_string = f"explain sbic sentence: {post}"
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"  
    elif io_format == 't5_fewshot_infilling':
        input_string = f"explain sbic sentence: {post} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':
        answer_bool = 'True' if item["offensiveYN"] == 'offensive' else 'False'
        input_string = f"explain sbic sentence: {post} It is <extra_id_0> that this is offensive {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_more_natural':
        input_string = f"explain sbic sentence: {post} This is <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer} <extra_id_1> {abstr_expl} <extra_id_2>"
    elif io_format == 'squad_yn': 
        input_string = f"explain sbic question: Is this offensive? context: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_yn_with_tags': 
        input_string = f"explain sbic question: Is this offensive? context: post: {post}" # explain sbic question: Is this offensive? context: It's only happened to you like two times.
        answer_yn = unified_qa_sbic_label_mapping[item["offensiveYN"]] 
        answer_string = f"{answer_yn} {explanation_sep} {abstr_expl}" #  No because this post does not imply anything offensive
    elif io_format == 'squad_what': 
        input_string = f"explain sbic question: What is this? context: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'squad_what_with_tags': 
        input_string = f"explain sbic question: What is this? context: post: {post}" # explain sbic question: This is what? context: It's only happened to you like two times.
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")        
        answer_string = f"{answer} {explanation_sep} {abstr_expl}" # not_offensive because this post does not imply anything offensive    
    elif io_format == 'unified_qa_yn':
        input_string = f'explain is this offensive? \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}" 
    elif io_format == 'unified_qa_yn_with_tags':
        input_string = f'explain is this offensive? \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_yn_with_choices':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_yn_with_choices_and_tags':
        input_string = f'explain is this offensive? \\n (A) yes (B) no \\n post: {post.lower()}'
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what':
        input_string = f'explain what is this? \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_tags':
        input_string = f'explain what is this? \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unified_qa_what_with_choices':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"
    elif io_format == 'unified_qa_what_with_choices_and_tags':
        input_string = f'explain what is this? \\n (A) offensive (B) not_offensive \\n post: {post.lower()}'
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl.lower()}"  
    elif io_format == 'unifiedqa_unifew':
        input_string = f"Topic? \\n (A) offensive (B) not_offensive \\n {post}"
        answer = item["offensiveYN"].replace("not offensive", "not_offensive")
        answer_string = f"{answer} {explanation_sep} {abstr_expl}"
    else:
        raise ValueError("The IO format is not supported. Choose `standard` or `masked_cause_generate`.")

    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

def sensemaking_formatting(item, io_format, explanation_sep):
    # TODO: explore whether removing periods makes difference? 
    sent0 = item['sent0']
    sent1 = item['sent1']
    nonsensical_sentence = str(int(item['label'])+1)
    explanation = item['explanation'].lower()

    if io_format == 'copa_with_question':
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} question: nonsensical"
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}"
    elif io_format == 'copa_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} Less common is choice2"
        answer_string = f"{answer_bool} {explanation_sep} {explanation}"
    elif io_format == 't5_fewshot_infilling':  
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} <extra_id_0> {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> choice{nonsensical_sentence} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == 't5_fewshot_infilling_bool':  
        answer_bool = str(bool(int(item['label']))) # True if choice2 is more nonsensical    
        input_string = f"explain sensemaking choice1: {sent0} choice2: {sent1} It is <extra_id_0> that choice2 is less common {explanation_sep} <extra_id_1>"
        answer_string = f"<extra_id_0> {answer_bool} <extra_id_1> {explanation} <extra_id_2>"
    elif io_format == "squad_yn": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_yn_no_tags": 
        input_string = f"explain sensemaking question: Is choice2 more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer = "Yes" if bool(int(item['label'])) else "No"
        answer_string = f"{answer} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: choice1: {sent0} choice2: {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "squad_what_no_tags": 
        input_string = f"explain sensemaking question: What is more nonsensical? context: {sent0} {sent1}" # explain sensemaking question: What is nonsensical, choice1 or choice2? context: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" #  choice1 because state flowers are unique to each state.  
    elif io_format == "record": 
        input_string = f"explain sensemaking query: What is more nonsensical? entities: choice1, choice2 passage: choice1: {sent0} choice2: {sent1}" # explain sensemaking query: What is nonsensical? entities: choice1, choice2 passage: choice1: All state flowers are the scarlet carnation. choice2: The New Jersey state flower is the scarlet carnation.
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation}" # choice1 because state flowers are unique to each state.
    elif io_format == 'unifiedqa_yn_with_choices':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n (A) yes (B) no \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}" 
    elif io_format == 'unifiedqa_yn':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_yn_no_tags':
        answer = "yes" if bool(int(item['label'])) else "no"
        input_string = f'explain is choice2 more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"{answer} {explanation_sep} {explanation.lower()}"  
    elif io_format == 'unifiedqa_what_with_choices':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n (A) choice1 (B) choice2 \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n choice1: {sent0.lower()} choice2: {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "
    elif io_format == 'unifiedqa_what_no_tags':
        nonsensical_sentence = str(int(item['label'])+1)
        input_string = f'explain what is more nonsensical? \\n {sent0.lower()} {sent1.lower()}'
        answer_string = f"choice{nonsensical_sentence} {explanation_sep} {explanation.lower()}"  # use " BECAUSE "


    input_string = ' '.join(input_string.split())
    answer_string = ' '.join(answer_string.split())
    return input_string, answer_string

In [4]:
og_start_time = time.time()

#parser = HfArgumentParser(
#    (ModelArguments, DataTrainingArguments, TrainingArguments)
#)
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)

model_args, data_args, training_args, unused_args = parser.parse_args_into_dataclasses(
    ["--model_type", "t5-base",
     "--tokenizer_name", "t5-base",
     "--task_name", "esnli", 
     "--output_dir", "./esnli_output_t5_base", 
     "--n_shots", "10",
     "--do_train", "True"], return_remaining_strings=True)
# training_args.set_device = "cpu"
if unused_args != []:
    raise ValueError(f"Received unused arguments: {unused_args}")
# make sure only one dataset split pick if manually specifying evaluation file

if model_args.use_gpt3:
    assert training_args.do_train
    assert not training_args.do_eval
    assert data_args.generations_filepath is None
    if data_args.gpt3_max_eval_size is not None:
        assert data_args.gpt3_max_eval_size <= data_args.fewshot_eval_size
        assert data_args.gpt3_max_eval_size % 2 == 0
        assert data_args.gpt3_max_eval_size % 3 == 0

if data_args.generations_filepath is not None:
    training_args.do_train = False
    training_args.do_eval = False
    if "train" in data_args.generations_filepath:
        data_args.train_predict = True
        data_args.test_predict = False
        data_args.dev_predict = False
    elif "test" in data_args.generations_filepath:
        data_args.train_predict = False
        data_args.test_predict = True
        data_args.dev_predict = False
    elif "validation" in data_args.generations_filepath:
        data_args.train_predict = False
        data_args.test_predict = False
        data_args.dev_predict = True

if not training_args.do_train and data_args.generations_filepath is None:
    if not model_args.pretrained_model_file:
        raise Exception(
            "if not training a model from scratch, must specify a trained model to load for evaluation"
        )

if training_args.do_train:
    # create a save directory and a logfile
    training_args.output_dir = os.path.join(
        training_args.output_dir, datetime.now().strftime("%m%d%y_%H%M%S")
    )
    training_args.logging_dir = training_args.output_dir
    assert not os.path.exists(training_args.output_dir)
    os.makedirs(training_args.output_dir)

    if (
            os.path.exists(training_args.output_dir)
            and os.listdir(training_args.output_dir)
            and training_args.do_train
            and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )
    handlers = [
        logging.FileHandler(os.path.join(training_args.output_dir, "logger.log")),
        logging.StreamHandler(),
    ]
else:
    # don't overwrite existing logfile or create new directory
    training_args.output_dir = model_args.pretrained_model_file
    handlers = [logging.StreamHandler()]

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    handlers=handlers,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Save path: %s" % training_args.output_dir)

# get git hash and branch where deployed
repo = git.Repo(search_parent_directories=True)
git_hash = repo.head.object.hexsha
git_branch = repo.active_branch.name
logger.info("Git branch: %s" % git_branch)
logger.info("Git hash: %s" % git_hash)

model_class = "t5"
assert data_args.task_name in {"cos_e", "esnli", "sbic", "sensemaking", "ecqa"}

if training_args.do_train:
    # write command and args to file
    with open(
            os.path.join(training_args.output_dir, "commandline_args.txt"), "w"
    ) as f:
        f.write("Git branch: " + git_branch + "\n")
        f.write("Git hash: " + git_hash + "\n")
        f.write("Command:\n")
        f.write("\n".join(sys.argv[1:]))

# Set seed
set_seed(training_args.seed)
set_other_seeds(training_args.seed)

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
02/01/2023 20:31:23 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
02/01/2023 20:31:23 - INFO - __main__ -   Save path: ./esnli_output_t5_base/020123_203123
02/01/2023 20:31:24 - INFO - __main__ -   Git branch: dev
02/01/2023 20:31:24 - INFO - __main__ -   Git hash: 1cbb5c3b4e53baf31cbafc20d9655c63f091f901


In [5]:
# training_args.set_device = "cpu"

In [6]:
import logging
logger = logging.getLogger(__name__)
CONFIG_MAPPING = {"t5": T5Config}
MODEL_MAPPING = {"t5": T5ForConditionalGeneration}
TOKENIZER_MAPPING = {"t5": T5Tokenizer}
model_class = "t5"
tokenizer_name = TOKENIZER_MAPPING[model_class]
logger.info("Loading pretrained tokenizer...")
model_args.tokenizer_name='t5-base'
tokenizer = tokenizer_name.from_pretrained(model_args.tokenizer_name)#, cache_dir=model_args.cache_dir)

model = T5ForConditionalGeneration.from_pretrained("t5-base")

02/01/2023 20:31:24 - INFO - __main__ -   Loading pretrained tokenizer...
loading file https://huggingface.co/t5-base/resolve/main/spiece.model from cache at /home/huangyongfeng/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d
loading file https://huggingface.co/t5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/t5-base/resolve/main/tokenizer.json from cache at /home/huangyongfeng/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /ho

In [7]:
data_splits = {'train': None, 'validation': None, 'test': None}
original_data_splits = {'train': None, 'validation': None, 'test': None}  
data_args.io_format="t5_fewshot_infilling_without_choices_use_refined_expl"
data_args

DataTrainingArguments(task_name='esnli', early_stopping_patience=10, overwrite_cache=False, train_predict=False, test_predict=False, dev_predict=False, version_name='v1.11', generations_filepath=None, n_shots=10, fewshot_eval_size=350, io_format='t5_fewshot_infilling_without_choices_use_refined_expl', explanation_sep='explanation', data_path=None, gpt3_max_eval_size=None)

In [8]:
dataset = datasets.load_dataset(data_args.task_name)
# train_ids_list=[x['id'] for x in dataset["train"]]



  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
dataset.keys()
len(dataset['train']), len(dataset['validation']), len(dataset['test'])
dataset['train'][0].keys()
all_data_dict = {}
for da in dataset['train']:
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str not in all_data_dict.keys():
        all_data_dict[key_str] = da
    else:
        print("*****")
        print(da)
        print(all_data_dict[key_str])
        print("*****")

*****
{'premise': 'Child in red and blue shirt painting a log.', 'hypothesis': 'The child is painting.', 'label': 0, 'explanation_1': '"Child is painting" is a simpler version of  "child painting a log".', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Child in red and blue shirt painting a log.', 'hypothesis': 'The child is painting.', 'label': 0, 'explanation_1': 'Both subjects are painting', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Two small dogs run across the green grass.', 'hypothesis': 'Two dogs are outside.', 'label': 0, 'explanation_1': 'Dogs can be small.  Grass is usually outside.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Two small dogs run across the green grass.', 'hypothesis': 'Two dogs are outside.', 'label': 0, 'explanation_1': 'Dogs can be small. Grass is usually outside,', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A young woman is drawing with a Sharpie marker.', 'hypothesis': 'A woman is drawin

*****
{'premise': 'An attractive girl with a cigarette in one hand leaning over and showing lots of cleavage.', 'hypothesis': 'A girl is holding a cigarette.', 'label': 0, 'explanation_1': 'Holding something is done in one hand.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'An attractive girl with a cigarette in one hand leaning over and showing lots of cleavage.', 'hypothesis': 'A girl is holding a cigarette.', 'label': 0, 'explanation_1': 'Holding something is done in one hand.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Two race cars speed around a track.', 'hypothesis': 'Cars go around a track.', 'label': 0, 'explanation_1': 'rACE CARS SPEED ON THE TRACK.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Two race cars speed around a track.', 'hypothesis': 'Cars go around a track.', 'label': 0, 'explanation_1': 'Race cars go around a track..', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'An older man wearing a black ja

*****
{'premise': 'a man walking down an alleyway', 'hypothesis': 'A man is walking.', 'label': 1, 'explanation_1': 'We actually do know that the man is walking. That statement can be inferred.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'a man walking down an alleyway', 'hypothesis': 'A man is walking.', 'label': 0, 'explanation_1': 'Walking down an alleyway is the same as walking.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A woman walks up a path that cuts through cultivated plots while balancing straw on her head.', 'hypothesis': 'The woman is outdoors.', 'label': 0, 'explanation_1': 'A woman walks up a path is the same as being outdoors.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A woman walks up a path that cuts through cultivated plots while balancing straw on her head.', 'hypothesis': 'The woman is outdoors.', 'label': 0, 'explanation_1': 'A woman walks up a path is the same as being outdoors.', 'explanation_2': '', 'explanatio

*****
{'premise': 'A bunch of birds outside of a building.', 'hypothesis': 'The birds are outside.', 'label': 0, 'explanation_1': '"Outside" is a less detailed restatement of "outside a building".', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A bunch of birds outside of a building.', 'hypothesis': 'The birds are outside.', 'label': 0, 'explanation_1': 'Birds and a bunch of birds are synonyms.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A group of young African American kids sitting on a white wall in front of a white building.', 'hypothesis': 'A group of kids are outside.', 'label': 0, 'explanation_1': 'If the group of kids are sitting on a white wall in front of a white building, they are sitting outside.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A group of young African American kids sitting on a white wall in front of a white building.', 'hypothesis': 'A group of kids are outside.', 'label': 0, 'explanation_1': 'young African Americ

*****
{'premise': 'A man with black hair is wearing a red and yellow shirt and has a yellow clown nose and face makeup on.', 'hypothesis': 'A man is dressed as a clown.', 'label': 0, 'explanation_1': 'The red and yellow shirt, yellow clown nose and face makeup are descriptive words in the first sentence that are attributes of the man being dressed as a clown.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A man with black hair is wearing a red and yellow shirt and has a yellow clown nose and face makeup on.', 'hypothesis': 'A man is dressed as a clown.', 'label': 1, 'explanation_1': "Just because a man is wearing a red and yellow shirt doesn't necessarily imply he is dressed as a clown.", 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A person is jumping up in the woods, they are wearing a red sweater and yellow pants, they are surrounded by trees and leaves.', 'hypothesis': 'A person is jumping in the woods.', 'label': 0, 'explanation_1': 'Jumping usually

...

*****
{'premise': 'A jockey in blue on a race horse trails behind the jockey in orange.', 'hypothesis': 'A jockey is on a horse.', 'label': 0, 'explanation_1': 'jockey on a race horse trails implies that he is on a horse.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A jockey in blue on a race horse trails behind the jockey in orange.', 'hypothesis': 'A jockey is on a horse.', 'label': 0, 'explanation_1': 'race horse is inferred as  horse.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A band is performing on stage.', 'hypothesis': 'The stage is empty.', 'label': 2, 'explanation_1': "The band can't be performing on stage if the stage is empty.", 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A band is performing on stage.', 'hypothesis': 'The stage is empty.', 'label': 2, 'explanation_1': 'when a band is performing the stage could not be empty', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Two guys barbecuing, one in blue s

*****
{'premise': 'Two women with white head garb and long dresses sitting in front of a door on steps reading and drinking a beverage.', 'hypothesis': 'Two women are sitting behind their husbands as they wait for the kids to get home from school.', 'label': 1, 'explanation_1': 'No indication of them sitting behind their husbands.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Two women with white head garb and long dresses sitting in front of a door on steps reading and drinking a beverage.', 'hypothesis': 'Two women are sitting behind their husbands as they wait for the kids to get home from school.', 'label': 1, 'explanation_1': 'Not all woman have husbands. Just because they are sitting in fron of a door does not imply they are waiting for their kids to get home from school.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Two women with white head garb and long dresses sitting in front of a door on steps reading and drinking a beverage.', 'hypothesis'

*****
{'premise': 'A man with a beard who wears a black knit hat and a gray vest stands by a building.', 'hypothesis': 'A man is standing by a building.', 'label': 0, 'explanation_1': 'Standing is a synonym of stands.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'A man with a beard who wears a black knit hat and a gray vest stands by a building.', 'hypothesis': 'A man is standing by a building.', 'label': 0, 'explanation_1': '"A man with a beard who wears a black knit hat and a gray vest stands by a building" is a more detailed rephrase of "a man is standing by a building."', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'An elderly women is sitting and leaning against a wall, holding a photo and a plastic glass.', 'hypothesis': 'Woman leaning against wall', 'label': 0, 'explanation_1': 'the woman is leaning against a wall, old or not.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'An elderly women is sitting and leaning against a wall, holding 

*****
{'premise': 'Two kids are looking up, one has no shirt and the other has on a white shirt with a stool on his head.', 'hypothesis': 'Two kids are looking down.', 'label': 2, 'explanation_1': 'One cannot be looking up and looking down simultaneously.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Two kids are looking up, one has no shirt and the other has on a white shirt with a stool on his head.', 'hypothesis': 'Two kids are looking down.', 'label': 2, 'explanation_1': 'One cannot be looking up and looking down simultaneously.', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Two dogs splash through the water.', 'hypothesis': 'Two dogs are getting wet.', 'label': 0, 'explanation_1': 'If the dogs Splash through the water than the dogs are getting wet because splashing causes wetness and wetting.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Two dogs splash through the water.', 'hypothesis': 'Two dogs are getting wet.', 'label': 0, 'explanat

*****
{'premise': 'Boys are selling flowers, drinks, mugs, and other items on the sidewalk.', 'hypothesis': 'Boys are selling flowers.', 'label': 0, 'explanation_1': 'Selling flowers on the sidewalk is a way of selling flowers.', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Boys are selling flowers, drinks, mugs, and other items on the sidewalk.', 'hypothesis': 'Boys are selling flowers.', 'label': 0, 'explanation_1': 'Flowers is one of the items the boys are selling', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'Man dressed in drag pointing at his open mouth.', 'hypothesis': 'A man is dressed in drag.', 'label': 0, 'explanation_1': 'A man is dressed in drag is a simple form of sentence 1', 'explanation_2': '', 'explanation_3': ''}
{'premise': 'Man dressed in drag pointing at his open mouth.', 'hypothesis': 'A man is dressed in drag.', 'label': 0, 'explanation_1': 'drag pointing at his open mouth shows that drag', 'explanation_2': '', 'explanation_3': ''

In [10]:
for da in dataset['validation']:
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str not in all_data_dict.keys():
        all_data_dict[key_str] = da
    else:
        print("*****")
        print(da)
        print(all_data_dict[key_str])
        print("*****")

*****
{'premise': 'A person is surfing on a large wave.', 'hypothesis': 'A person is surfing.', 'label': 0, 'explanation_1': 'There are both a person surfing in the scene.', 'explanation_2': 'Surfing on a large wave is a type of surfing.', 'explanation_3': 'A person surfing on a large wave, is surfing.'}
{'premise': 'A person is surfing on a large wave.', 'hypothesis': 'A person is surfing.', 'label': 0, 'explanation_1': 'person is surfing simply means person is surfing', 'explanation_2': '', 'explanation_3': ''}
*****
*****
{'premise': 'A dog carrying a stick in its mouth runs through a snow-covered field.', 'hypothesis': 'A animal carried a stick through a snow covered field.', 'label': 0, 'explanation_1': 'A dog is an animal.', 'explanation_2': 'Dog is animal, and carried a stick implies in its mouth.', 'explanation_3': 'a dog is an animal'}
{'premise': 'A dog carrying a stick in its mouth runs through a snow-covered field.', 'hypothesis': 'A animal carried a stick through a snow co

In [11]:
for da in dataset['test']:
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str not in all_data_dict.keys():
        all_data_dict[key_str] = da
    else:
        print("*****")
        print(da)
        print(all_data_dict[key_str])
        print("*****")

In [12]:
len(all_data_dict), len(dataset['train']) + len(dataset['validation'])+ len(dataset['test'])


(568404, 569033)

In [13]:
#rationale generation labeled data construction 115
import pandas as pd
scr_csqa_labeled_path="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/handwritten_snli_examples.csv"
scr_csqa_label_df=pd.read_csv(scr_csqa_labeled_path)
scr_csqa_label_data=datasets.load_dataset('csv', data_files=scr_csqa_labeled_path)
scr_csqa_label_data['train'][0].keys()

refine_train_data=[]
for da in scr_csqa_label_data['train']:
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str in all_data_dict.keys():
        new_da=all_data_dict[key_str]
        new_da['our_explanation'] = da['our_explanation']
        refine_train_data.append(new_da)
        
    
# for kk, (ex,da) in enumerate(zip(scr_csqa_label_data, data_splits['train'])):
# #     print(da)
#     data_splits['train'][kk]['our_explanation']=ex
#     #print(type(data_splits['train'][kk]),data_splits['train'][kk].keys())
#     da['our_explanation']=ex
#     refine_train_data.append(da)

# len(data_splits['train'])


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
#rationale unlabeled data construction 991
import pdb
scr_csqa_unlabeled_test_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/snli_train.csv"
fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=scr_csqa_unlabeled_test_file)
scr_csqa_unlabeled_test_df=pd.read_csv(scr_csqa_unlabeled_test_file)
fse_csqa_dev_data_dict={}
for kk, da in enumerate(fse_csqa_dev_dataset['train']):
    #pdb.set_trace()
    id_=da['Input.id']
    if da['Answer.acceptable']:
        answer_accept=set(da['Answer.acceptable'].split('|'))
    else:
        answer_accept=set()
    explanation_list=[da['Input.explanation_1'], 
                      da['Input.explanation_2'],
                      da['Input.explanation_3'],
                      da['Input.explanation_4'],
                      da['Input.explanation_5']]
    if id_ not in fse_csqa_dev_data_dict.keys():
        fse_csqa_dev_data_dict[id_]={"index":kk,"id":id_,
                                     "premise":da["Input.premise"],
                                     "hypothesis":da["Input.hypothesis"],
                                     'answer':da['Input.gold_label'],
                                     "accept_set_list":[answer_accept],
                                     "explanation_list":explanation_list}
        #[[kk, id_, answer_accept, explanation_list]]
    else:
        fse_csqa_dev_data_dict[id_]["accept_set_list"].append(answer_accept)
    if len(fse_csqa_dev_data_dict[id_]["accept_set_list"])==3:
        fse_csqa_dev_data_dict[id_]["common_expl_list"]=[]
        common_accept_expl_sample=set.intersection(fse_csqa_dev_data_dict[id_]["accept_set_list"][0], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][1], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][2])
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            fse_csqa_dev_data_dict[id_]["common_expl_list"].append(fse_csqa_dev_data_dict[id_]["explanation_list"][idx])
            
# fse_csqa_dev_data_dict['snli_train_570']
refine_dev_data=[]
for kk, da in fse_csqa_dev_data_dict.items():
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str in all_data_dict.keys() and da["common_expl_list"]:
        new_da=all_data_dict[key_str]
        our_expl = ""
        for expl in da["common_expl_list"]:
            if len(expl) > len(our_expl):
                our_expl=expl
        new_da['our_explanation'] = our_expl#da['our_explanation']
        new_da['common_expl_list'] = da["common_expl_list"]
        refine_dev_data.append(new_da)
        #print(len(da["common_expl_list"]))
len(refine_dev_data)
    

  0%|          | 0/1 [00:00<?, ?it/s]

509

In [15]:
#rationale unlabeled data construction 991
import pdb
scr_csqa_unlabeled_test_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/snli_test.csv"
fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=scr_csqa_unlabeled_test_file)
scr_csqa_unlabeled_test_df=pd.read_csv(scr_csqa_unlabeled_test_file)
fse_csqa_dev_data_dict={}
for kk, da in enumerate(fse_csqa_dev_dataset['train']):
    #pdb.set_trace()
    id_=da['Input.id']
    if da['Answer.acceptable']:
        answer_accept=set(da['Answer.acceptable'].split('|'))
    else:
        answer_accept=set()
    explanation_list=[da['Input.explanation_1'], 
                      da['Input.explanation_2'],
                      da['Input.explanation_3'],
                      da['Input.explanation_4'],
                      da['Input.explanation_5']]
    if id_ not in fse_csqa_dev_data_dict.keys():
        fse_csqa_dev_data_dict[id_]={"index":kk,"id":id_,
                                     "premise":da["Input.premise"],
                                     "hypothesis":da["Input.hypothesis"],
                                     'answer':da['Input.gold_label'],
                                     "accept_set_list":[answer_accept],
                                     "explanation_list":explanation_list}
        #[[kk, id_, answer_accept, explanation_list]]
    else:
        fse_csqa_dev_data_dict[id_]["accept_set_list"].append(answer_accept)
    if len(fse_csqa_dev_data_dict[id_]["accept_set_list"])==3:
        fse_csqa_dev_data_dict[id_]["common_expl_list"]=[]
        common_accept_expl_sample=set.intersection(fse_csqa_dev_data_dict[id_]["accept_set_list"][0], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][1], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][2])
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            fse_csqa_dev_data_dict[id_]["common_expl_list"].append(fse_csqa_dev_data_dict[id_]["explanation_list"][idx])
            
# fse_csqa_dev_data_dict['snli_train_570']
#refine_test_data=[]
for kk, da in fse_csqa_dev_data_dict.items():
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str in all_data_dict.keys() and da["common_expl_list"]:
        new_da=all_data_dict[key_str]
        our_expl = ""
        for expl in da["common_expl_list"]:
            if len(expl) > len(our_expl):
                our_expl=expl
        new_da['our_explanation'] = our_expl#da['our_explanation']
        new_da['common_expl_list'] = da["common_expl_list"]
        refine_dev_data.append(new_da)
        #print(len(da["common_expl_list"]))
len(refine_dev_data)
    

  0%|          | 0/1 [00:00<?, ?it/s]

640

In [16]:
#rationale unlabeled data construction 991
import pdb
scr_csqa_unlabeled_test_file="/cognitive_comp/huangyongfeng/evaluate_LM_with_rationalization/few_shot_explanations/data/acceptability_annotations/snli_test.csv"
fse_csqa_dev_dataset = datasets.load_dataset('csv', data_files=scr_csqa_unlabeled_test_file)
scr_csqa_unlabeled_test_df=pd.read_csv(scr_csqa_unlabeled_test_file)
fse_csqa_dev_data_dict={}
for kk, da in enumerate(fse_csqa_dev_dataset['train']):
    #pdb.set_trace()
    id_=da['Input.id']
    if da['Answer.acceptable']:
        answer_accept=set(da['Answer.acceptable'].split('|'))
    else:
        answer_accept=set()
    explanation_list=[da['Input.explanation_1'], 
                      da['Input.explanation_2'],
                      da['Input.explanation_3'],
                      da['Input.explanation_4'],
                      da['Input.explanation_5']]
    if id_ not in fse_csqa_dev_data_dict.keys():
        fse_csqa_dev_data_dict[id_]={"index":kk,"id":id_,
                                     "premise":da["Input.premise"],
                                     "hypothesis":da["Input.hypothesis"],
                                     'answer':da['Input.gold_label'],
                                     "accept_set_list":[answer_accept],
                                     "explanation_list":explanation_list}
        #[[kk, id_, answer_accept, explanation_list]]
    else:
        fse_csqa_dev_data_dict[id_]["accept_set_list"].append(answer_accept)
    if len(fse_csqa_dev_data_dict[id_]["accept_set_list"])==3:
        fse_csqa_dev_data_dict[id_]["common_expl_list"]=[]
        common_accept_expl_sample=set.intersection(fse_csqa_dev_data_dict[id_]["accept_set_list"][0], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][1], 
                                                   fse_csqa_dev_data_dict[id_]["accept_set_list"][2])
        for idx in list(common_accept_expl_sample):
            idx=int(idx)-1
            fse_csqa_dev_data_dict[id_]["common_expl_list"].append(fse_csqa_dev_data_dict[id_]["explanation_list"][idx])
            
# fse_csqa_dev_data_dict['snli_train_570']
refine_test_data=[]
for kk, da in fse_csqa_dev_data_dict.items():
    hyp=da['hypothesis']
    pre=da['premise']
    key_str = f'hypothesis: {hyp}; premise:{pre}'
    if key_str in all_data_dict.keys() and da["common_expl_list"]:
        new_da=all_data_dict[key_str]
        our_expl = ""
        for expl in da["common_expl_list"]:
            if len(expl) > len(our_expl):
                our_expl=expl
        new_da['our_explanation'] = our_expl#da['our_explanation']
        new_da['common_expl_list'] = da["common_expl_list"]
        refine_test_data.append(new_da)
        #print(len(da["common_expl_list"]))
len(refine_test_data)
    

  0%|          | 0/1 [00:00<?, ?it/s]

131

In [17]:
our_data_splits={}
our_data_splits['train']=refine_train_data
our_data_splits['dev']=refine_dev_data
our_data_splits['test']=refine_test_data
refine_dev_data[0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3', 'our_explanation', 'common_expl_list'])

# shots number

In [18]:
chatgpt_rationales = ["sample 0: explain why the relation is neutral between hypothesis: they are smiling at their parents and premise: children smiling and waving at camera</s>",
"The relation between the hypothesis \"they are smiling at their parents\" and the premise \"children smiling and waving at camera\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that children are smiling and waving at the camera, but it does not specify who they are smiling and waving at, so it is possible that they are smiling and waving at their parents, but it is also possible that they are smiling and waving at someone else. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 1: explain why the relation is contradiction between hypothesis: two people are playing cribbage. and premise: two men in safety vests walk down the road.</s>",
"The relation between the hypothesis \"two people are playing cribbage\" and the premise \"two men in safety vests walk down the road\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that two people are playing cribbage, which implies they are in a stationary location, most likely indoors, and are engaged in a game. On the other hand, the premise states that two men are walking down the road, which implies that they are in motion, and most likely outdoors. These two statements cannot both be true at the same time as the people described in the hypothesis are not doing the same activity as the men described in the premise. Therefore, the relation between these two statements is a contradiction.",





"sample 2: explain why the relation is neutral between hypothesis: the tourists are getting a gondola ride. and premise: the tourists are out on the river in italy, the man in the black and white striped shirt will tell them about the seine.</s>",
"The relation between the hypothesis \"the tourists are getting a gondola ride\" and the premise \"the tourists are out on the river in Italy, the man in the black and white striped shirt will tell them about the Seine\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that the tourists are out on the river in Italy and that a man in a black and white striped shirt will tell them about the Seine, but it does not specify what the tourists are doing or where they are exactly. The premise does not specify if they are in a gondola or not, so it is possible that they are getting a gondola ride, but it is also possible that they are doing something else. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 3: explain why the relation is neutral between hypothesis: a person is training his horse for a competition. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation between the hypothesis \"a person is training his horse for a competition\" and the premise \"a person on a horse jumps over a broken down airplane\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that a person on a horse jumps over a broken down airplane, but it does not specify what the person and horse are doing. It could be that they are training for a competition, but it could also be that they're just doing some exercise or practicing jumping or obstacle course. The premise does not indicate if the person is training the horse for a competition or not. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",



"sample 4: explain why the relation is contradiction between hypothesis: the man is in a desert. and premise: an man in the north pole walking on ice.</s>",
"The relation between the hypothesis \"the man is in a desert\" and the premise \"a man in the North Pole walking on ice\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that the man is in a desert, which is a hot and dry area with little vegetation, while the premise states that the man is in the North Pole, which is a cold and icy area with no vegetation. These two statements cannot both be true at the same time as they describe two completely different environments and conditions. Therefore, the relation between these two statements is a contradiction.",




"sample 5: explain why the relation is entailment between hypothesis: a woman ropes an animal. and premise: a woman on the back of a horse ropes a cattle.</s>",
"The relation between the hypothesis \"a woman ropes an animal\" and the premise \"a woman on the back of a horse ropes a cattle\" is entailment because the premise confirms the hypothesis and provides additional information. The hypothesis states that a woman ropes an animal, which is a general statement, and the premise provides specific information that confirms the general statement, that a woman on the back of a horse ropes a cattle. The premise specifies that the woman is on the back of a horse and she ropes a cattle, which is an animal, which is the general statement that the hypothesis makes. The premise provides additional information, the woman is on the back of a horse and the animal she ropes is a cattle which is specific and confirms the general statement of the hypothesis. Therefore, the relation between these two statements is entailment.",




"sample 6: explain why the relation is contradiction between hypothesis: sports aren't played they are worn and premise: a young lacrosse player is catching his breath.</s>",
"The relation between the hypothesis \"Sports aren't played they are worn\" and the premise \"A young lacrosse player is catching his breath\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that sports are not played, they are worn, which implies that they are not an activity but rather a clothing or gear. On the other hand, the premise states that a young lacrosse player is catching his breath, which implies that he is engaged in an activity that requires physical effort and exertion, that is playing lacrosse. These two statements cannot both be true at the same time as the lacrosse player is playing and not wearing it. Therefore, the relation between these two statements is a contradiction.",





"sample 7: explain why the relation is neutral between hypothesis: a surgeon performs open heart surgery. and premise: a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs.</s>",
"The relation between the hypothesis \"a surgeon performs open heart surgery\" and the premise \"a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that a surgeon performs open heart surgery, which is a specific type of surgery, and the premise provides information about a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs. The premise does not specify what type of surgery is being performed, and it only describes the attire and appearance of the surgeons and the fact that they are performing surgery. It does not indicate whether the surgery is open heart surgery or not. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",





"sample 8: explain why the relation is neutral between hypothesis: a man awaits the results of his hiv test. and premise: a man in a blue long-sleeved jacket crying out of fear.</s>",
"The relation between the hypothesis \"a man awaits the results of his HIV test\" and the premise \"a man in a blue long-sleeved jacket crying out of fear\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that a man awaits the results of his HIV test, which implies that the man is waiting for the results of a medical test for HIV. The premise states that a man in a blue long-sleeved jacket crying out of fear, which implies that the man is experiencing fear and is crying. The premise does not indicate whether the man is waiting for the results of an HIV test or not, and it only describes the attire and appearance of the man and his emotional state. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 9: explain why the relation is neutral between hypothesis: the people are trying to get their ball back. and premise: people holding onto ropes while trying to get something through a window on the second floor.</s>",
"The relation between the hypothesis \"The people are trying to get their ball back\" and the premise \"people holding onto ropes while trying to get something through a window on the second floor\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that the people are trying to get their ball back, which implies they have lost a ball and they are trying to retrieve it. The premise describes people holding onto ropes while trying to get something through a window on the second floor, which implies that the people are performing some sort of activity involving ropes and a window on the second floor. The premise does not indicate whether they are trying to get a ball back or not, and it only describes the activity that people are doing and the context. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 10: explain why the relation is entailment between hypothesis: he is happy. and premise: a boy in colorful pants plays guitar and laughs together with his friends.</s>",
"The relation between the hypothesis \"he is happy\" and the premise \"a boy in colorful pants plays guitar and laughs together with his friends\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"he is happy\" which implies that the person is in a positive emotional state. The premise describes a boy in colorful pants who is playing guitar and laughing together with his friends, which implies that the boy is enjoying himself and having a good time. Laughter and playing guitar with friends is a strong indication of happiness. Therefore, the premise confirms the hypothesis that \"he is happy\".",




"sample 11: explain why the relation is entailment between hypothesis: the people are outside. and premise: people are walking around a park area.</s>",
"The relation between the hypothesis \"the people are outside\" and the premise \"people are walking around a park area\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"the people are outside\", which implies that the people are in an open-air environment and not indoors. The premise describes people walking around a park area, which implies that the people are outside in a public park where people can walk and enjoy the nature. This confirms that the people are outside, as the park area is an open-air environment. Therefore, the premise confirms the hypothesis that \"the people are outside\".",




"sample 12: explain why the relation is entailment between hypothesis: two men are with animals. and premise: two men chase cattle on horses.</s>",
"The relation between the hypothesis \"two men are with animals\" and the premise \"two men chase cattle on horses\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"two men are with animals\", which implies that the two men are in the presence of or interacting with animals. The premise describes two men chasing cattle on horses, which implies that the two men are with animals as they are interacting with cattle, which are animals. Additionally, the fact that they are doing so on horses also confirms that they are with animals. Therefore, the premise confirms the hypothesis that \"two men are with animals\".",




"sample 13: explain why the relation is neutral between hypothesis: the two men play very well. and premise: two men are playing acoustic guitars.</s>",
"The relation between the hypothesis \"the two men play very well\" and the premise \"two men are playing acoustic guitars\" is neutral because the premise does not confirm or deny the hypothesis. The hypothesis states that \"the two men play very well,\" which implies that the two men have a high level of skill or proficiency in playing the guitars. The premise describes two men playing acoustic guitars, which does not confirm or deny the hypothesis. It is not specified whether the two men are playing well or not. It could be that they are playing well, poorly, or somewhere in between. The premise does not provide any information about the skill level of the men playing the guitars, making the relation neutral.",



"sample 14: explain why the relation is contradiction between hypothesis: an elderly woman bathes and premise: a child is laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone.</s>",
"The relation between the hypothesis \"an elderly woman bathes\" and the premise \"a child is laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone\" is contradiction because the premise denies the hypothesis. The hypothesis states that \"an elderly woman bathes,\" which implies that an elderly woman is in the process of cleaning herself with water. The premise describes a child laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone. This implies that the person in the bathtub is a child and not an elderly woman. This contradicts the hypothesis as the premise is describing a person that is not elderly and not a woman. Therefore, the premise denies the hypothesis that \"an elderly woman bathes\".",




"sample 15: explain why the relation is entailment between hypothesis: a person is outdoors, on a horse. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation between the hypothesis \"a person is outdoors, on a horse\" and the premise \"a person on a horse jumps over a broken down airplane\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"a person is outdoors, on a horse,\" which implies that the person is outside and mounted on a horse. The premise describes a person on a horse jumping over a broken down airplane, which confirms the hypothesis that the person is outside and mounted on a horse. The premise provides additional information about the person's location and activity, but it does not contradict the main idea that the person is outside and on a horse. Therefore, the premise confirms the hypothesis \"a person is outdoors, on a horse\".",




"sample 16: explain why the relation is entailment between hypothesis: a girl and a boy stick their tongues out. and premise: a girl with a little boy both rolling their tongue as they stick it out.</s>",
"The relation between the hypothesis \"a girl and a boy stick their tongues out\" and the premise \"a girl with a little boy both rolling their tongue as they stick it out\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"a girl and a boy stick their tongues out\" which implies that at least one girl and one boy are sticking their tongues out. The premise describes a girl with a little boy both rolling their tongue as they stick it out, which confirms the hypothesis that at least one girl and one boy are sticking their tongues out. The premise provides additional information about the specific girl and boy and their action, but it does not contradict the main idea that at least one girl and one boy are sticking their tongues out. Therefore, the premise confirms the hypothesis \"a girl and a boy stick their tongues out\"",





"sample 17: explain why the relation is neutral between hypothesis: a boy was showing off for a girl. and premise: a young boy wearing a tank-top is climbing a tree.</s>",
"The relation between the hypothesis \"a boy was showing off for a girl\" and the premise \"a young boy wearing a tank-top is climbing a tree\" is neutral because the premise doesn't provide enough information to confirm or contradict the hypothesis. The hypothesis states that the boy was showing off for a girl, which implies that there is a girl present and the boy is doing something to impress her. The premise describes a young boy wearing a tank-top climbing a tree, which doesn't provide enough information to confirm or contradict the hypothesis. The premise doesn't tell us if there is a girl present or not and doesn't tell us if the boy is doing something to impress her or not. Therefore, the premise doesn't give enough information to confirm or contradict the hypothesis \"a boy was showing off for a girl\" thus the relation is neutral.",





"sample 18: explain why the relation is contradiction between hypothesis: three women are walking together outside. and premise: three women sitting and waiting for something on a bench.</s>",
"The relation between the hypothesis \"three women are walking together outside\" and the premise \"three women sitting and waiting for something on a bench\" is contradiction because the premise contradicts the hypothesis. The hypothesis states that the three women are walking together outside, which implies that they are moving around and not stationary. The premise describes three women sitting and waiting for something on a bench, which contradicts the idea that they are moving around and walking. The premise clearly shows that the women are sitting and not walking, therefore it contradicts the hypothesis that states that the three women are walking together outside, thus the relation is contradiction.",




"sample 19: explain why the relation is entailment between hypothesis: a woman singing and premise: a woman with brown hair wearing a red polka dot dress is singing into a microphone.</s>",
"The relation between the hypothesis \"a woman singing\" and the premise \"a woman with brown hair wearing a red polka dot dress is singing into a microphone\" is entailment because the premise provides more information that supports the hypothesis. The hypothesis states that a woman is singing, which means that she is producing sound with her voice. The premise describes a woman with brown hair wearing a red polka dot dress who is singing into a microphone, which confirms the hypothesis that a woman is singing. It also gives additional information such as the woman's appearance, clothing and the tool she's using to produce the sound (microphone) which supports the hypothesis that a woman is singing. Therefore, the premise entails the hypothesis, making the relation entailment.",




"sample 20: explain why the relation is neutral between hypothesis: a singer sings their most famous song. and premise: a singer entertaining the crowd.</s>",
"The relation is neutral between the hypothesis \"a singer sings their most famous song\" and the premise \"a singer entertaining the crowd\" because the premise does not specify what song the singer is singing, so it could be their most famous song, but it could also be a different song. Therefore, the premise does not support or contradict the hypothesis.",





"sample 21: explain why the relation is neutral between hypothesis: a guy is running from the police. and premise: a guy in white shirt and black pants is running.</s>",
"The relation is neutral between the hypothesis \"a guy is running from the police\" and the premise \"a guy in white shirt and black pants is running\" because the premise only states that a guy in a certain clothing is running, it doesn't specify the reason why he is running, so it could be from the police or for another reason, the premise does not support or contradict the hypothesis.",




"sample 22: explain why the relation is contradiction between hypothesis: the man is working on his car and premise: women walking down the street in jeans, black high heel boots, and a beige handbag.</s>",
"The relation is a contradiction because the hypothesis states that the man is working on his car, which suggests that he is engaged in a specific activity related to automobiles. However, the premise describes a group of women who are walking down the street, which is not related to the man working on a car and thus contradicts the hypothesis.",




"sample 23: explain why the relation is entailment between hypothesis: a man wields a tool and premise: a man is cutting a fish.</s>",
"The relation is entailment between the hypothesis \"a man wields a tool\" and the premise \"a man is cutting a fish\" because the premise provides specific information that supports the hypothesis. The hypothesis states that a man is using a tool and the premise specifies that the man is cutting a fish, which requires the use of a tool such as a knife. Therefore, the premise confirms that the man is indeed using a tool, and thus the two statements are logically consistent.",





"sample 24: explain why the relation is neutral between hypothesis: a band is playing a concert. and premise: this band is playing on a stage.</s>",
"The relation is neutral between the hypothesis \"a band is playing a concert\" and the premise \"this band is playing on a stage\" because both the hypothesis and the premise mention the band playing, but do not provide any information that would indicate whether the band is playing a concert or not. The premise only states that the band is playing on a stage, which could be part of a concert, but it does not confirm or deny the hypothesis that the band is playing a concert.",




"sample 25: explain why the relation is contradiction between hypothesis: the horses are becoming suspicious of my apples. and premise: there is a red truck behind the horses.</s>",
"The relation is contradiction because the premise (\"there is a red truck behind the horses\") does not match the hypothesis (\"the horses are becoming suspicious of my apples\"). The horse's behavior is described as being suspicious of apples, not having any relation with a red truck. The two statements are not related and cannot both be true at the same time.",




"sample 26: explain why the relation is neutral between hypothesis: two african children are friends. and premise: two african children standing by a hut with their faces painted.</s>",
"The relation is neutral between the hypothesis \"two african children are friends\" and the premise \"two african children standing by a hut with their faces painted\" because the premise does not provide any information about the relationship between the children. It only describes their physical appearance and location. The hypothesis states that the children are friends, but the premise does not confirm or deny this.",





"sample 27: explain why the relation is contradiction between hypothesis: a nurse is pushing a surgical cart and premise: a woman wearing gloves working at a meat counter.</s>",
"The relation is a contradiction because the hypothesis and premise describe two very different scenarios. In the hypothesis, a nurse is described as pushing a surgical cart in a hospital setting, while in the premise, a woman is described as working at a meat counter in a grocery store. These two images are not consistent with each other and therefore, the relation is a contradiction.",





"sample 28: explain why the relation is entailment between hypothesis: there are children present and premise: children smiling and waving at camera</s>",
"The relation between the hypothesis \"there are children present\" and the premise \"children smiling and waving at camera\" is entailment because the premise provides evidence that supports the hypothesis. The presence of children smiling and waving in the premise confirms that there are indeed children present, as stated in the hypothesis. The hypothesis is entailed by the premise because the premise is providing visual evidence of the presence of children.",





"sample 29: explain why the relation is contradiction between hypothesis: a girl is sitting at the kitchen table eating lunch. and premise: a girl, about 8 years old, is sitting in front of a television, delighted with a stuffed animal, while a boy watches.</s>",
"The relation is contradiction because the hypothesis describes the girl as sitting at the kitchen table eating lunch, while the premise describes the girl as sitting in front of a television, delighted with a stuffed animal, which would suggest that she is not eating lunch at the kitchen table. These two statements cannot both be true at the same time, making the relation a contradiction.",




"sample 30: explain why the relation is neutral between hypothesis: the girls is having fun at the park. and premise: the girl wears a purple outfit and swings upside down.</s>",
"The relation is neutral between the hypothesis \"the girl is having fun at the park\" and the premise \"the girl wears a purple outfit and swings upside down\" because the premise only describes the girl's appearance and actions, but it does not provide any information about the girl's emotional state or the context in which she is participating in those actions. The hypothesis suggests that the girl is having fun, but the premise does not confirm or deny this. It is possible that the girl is having fun at the park, but it is also possible that she is not.",




"sample 31: explain why the relation is neutral between hypothesis: the people are having a business meeting. and premise: an aerial view of a group of meeting talking to each other.</s>",
"The relation between the hypothesis \"the people are having a business meeting\" and the premise \"an aerial view of a group of people talking to each other\" is neutral because the premise provides general information about a group of people talking to each other, but it does not specify that they are having a business meeting. The premise does not necessarily support or contradict the hypothesis.",




"sample 32: explain why the relation is entailment between hypothesis: a woman and dog are together. and premise: dog on a red leash with a woman.</s>",
"The relation is entailment between the hypothesis \"a woman and dog are together\" and the premise \"dog on a red leash with a woman\" because the premise provides specific details that confirm the hypothesis. The premise states that there is a dog on a red leash with a woman, which implies that the woman and dog are together as stated in the hypothesis. The information in the premise supports and confirms the information in the hypothesis.",




"sample 33: explain why the relation is neutral between hypothesis: soccer players play in the world championship in front of a sell out audience. and premise: soccer players playing a game in front of a big crowd.</s>",
"The relation is neutral because the premise provides no information about the context of the soccer game, such as whether it is a world championship or whether the audience is a sell-out crowd, while the hypothesis does. The premise only states that soccer players are playing a game in front of a big crowd, which is not necessarily enough information to determine whether the game is a world championship or not.",





"sample 34: explain why the relation is entailment between hypothesis: a child is holding a picture and premise: child holds up a picture of mahmoud ahmadinejad while men converse behind him.</s>",
"The relation is entailment between the hypothesis \"a child is holding a picture\" and the premise \"child holds up a picture of mahmoud ahmadinejad while men converse behind him\" because the premise provides specific details about the picture the child is holding, and that it is of a specific person, Mahmoud Ahmadinejad. This additional information is entailed by the more general information in the hypothesis that the child is holding a picture.",




"sample 35: explain why the relation is contradiction between hypothesis: there are people dancing in the rain. and premise: a man and two women are browsing through a market that sells vases, jewelery and other items.</s>",
"The relation between the hypothesis \"there are people dancing in the rain\" and the premise \"a man and two women are browsing through a market that sells vases, jewelry, and other items\" is a contradiction because the two statements describe completely different actions and locations. The hypothesis implies that people are actively engaging in a specific activity (dancing) in a specific weather condition (rain), while the premise describes a completely different activity (browsing through a market) and location (indoors). These two statements cannot both be true at the same time, which is why the relation between them is a contradiction.",




"sample 36: explain why the relation is entailment between hypothesis: four boys are outside. and premise: four boys play in the mud.</s>",
"The relation is entailment between the hypothesis \"four boys are outside\" and the premise \"four boys play in the mud\" because the premise is a specific example of the situation described in the hypothesis. If it is known that there are four boys outside, it must be true that they are playing in the mud, as the premise states. The premise provides additional information that is entailed by the hypothesis.",




"sample 37: explain why the relation is contradiction between hypothesis: two motorcylists are driving up a mountain. and premise: a motorcyclist drives up the side of a green mountain while another bike remains stationary.</s>",
"The relation is a contradiction because the hypothesis states that two motorcyclists are driving up a mountain, but the premise states that only one motorcyclist is driving while the other bike remains stationary. The two statements cannot both be true at the same time, as only one motorcyclist is seen to be moving in the premise, while the hypothesis states that both are moving.",




"sample 38: explain why the relation is entailment between hypothesis: the boy does a skateboarding trick. and premise: a boy is jumping on skateboard in the middle of a red bridge.</s>",
"The relation is entailment because the premise \"a boy is jumping on Skateboard in the middle of a red bridge\" describes an action that is consistent with the hypothesis \"the boy does a skateboarding trick.\" The premise provides additional details and context to the hypothesis, but the basic action of the boy performing a skateboarding trick is still present in the premise.",




"sample 39: explain why the relation is contradiction between hypothesis: the horses are in their stalls in the barn. and premise: several horses are running a race on a muddy track.</s>",
"The relation is contradiction because the hypothesis states that the horses are in their stalls in the barn, which implies that they are not moving or engaging in any activity. The premise, on the other hand, states that several horses are running a race on a muddy track, which is an activity that would not be possible if the horses were in their stalls in the barn. These two statements cannot both be true at the same time, and therefore, they contradict each other.",




"sample 40: explain why the relation is neutral between hypothesis: an giant collected flowers and premise: an giant egged shaped collection of flowers.</s>",
"The relation is neutral between the hypothesis \"an giant collected flowers\" and the premise \"an giant egged shaped collection of flowers\" because the premise does not provide any information about whether the flowers were collected by a giant or not. The premise simply describes an object made of flowers that has an egg-shaped form, without providing any information about the action or context. Therefore, the premise does not support or contradict the hypothesis, and the relation is neutral.",




"sample 41: explain why the relation is neutral between hypothesis: women are competing in a table tennis tournament at the beach. and premise: two females in bikinis are playing table tennis at the beach while another female in a towel watches.</s>",
"The relation is neutral because the hypothesis and the premise both describe women competing in a table tennis tournament at the beach, but the premise also includes additional details such as what they are wearing. The additional details do not contradict or support the statement in the hypothesis, so the relation is neutral.",




"sample 42: explain why the relation is contradiction between hypothesis: the table was wood. and premise: two men sit at a glass table with papers on it.</s>",
"The relation is contradiction because the hypothesis states that the table is made of wood, while the premise states that the table is made of glass. These two statements cannot both be true at the same time, as a table cannot be made of both wood and glass. This is an example of a logical contradiction, as it is impossible for both statements to be true.",




"sample 43: explain why the relation is entailment between hypothesis: a bearded man plays guitar. and premise: a long bearded man plays a furry guitar.</s>",
"The relation is entailment between the hypothesis \"a bearded man plays guitar\" and the premise \"a long bearded man plays a furry guitar\" because the premise provides additional information about the man's beard and the guitar that he is playing, but it does not change the fact that the man is playing a guitar, which is stated in the hypothesis. The premise is consistent with the hypothesis and adds more details to it.",




"sample 44: explain why the relation is entailment between hypothesis: the woman in knee-highs and jean shorts is walking down the sidewalk smoking. and premise: a woman in checkered knee-highs and jean shorts walks down a city sidewalk street while smoking a cigarette.</s>",
"The relation is entailment because the premise provides additional information that supports the hypothesis. The specific details provided in the premise, such as the woman wearing knee-highs and jean shorts, and smoking a cigarette, are consistent with and support the general statement made in the hypothesis that a woman is walking down the sidewalk smoking.",




"sample 45: explain why the relation is contradiction between hypothesis: a boy flips a burger. and premise: an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.</s>",
"The relation is contradiction between the hypothesis \"a boy flips a burger\" and the premise \"an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background\" because the hypothesis states that a specific action (flipping a burger) is happening in a specific setting (presumably a kitchen or grill area), while the premise describes a completely different scene with no indication of a burger-related activity. These two statements cannot both be true at the same time.",




"sample 46: explain why the relation is neutral between hypothesis: two couples are preparing dinner together. and premise: two ladies are cooking & talking and two gentlemen are talking in the opposite site of the kitchen.</s>",
"The relation between the hypothesis and premise is neutral because the premise describes two couples engaging in separate activities in the kitchen, while the hypothesis states that they are preparing dinner together. The premise does not provide enough information to confirm or deny the hypothesis.",





"sample 47: explain why the relation is entailment between hypothesis: a man runs outside past a building. and premise: a man wearing blue jogging pants with 3 white stripes and a t-shirt is running past a building with graffiti on it.</s>",
"The relation is entailment between the hypothesis \"a man runs outside past a building\" and the premise \"a man wearing blue jogging pants with 3 white stripes and a t-shirt is running past a building with graffiti on it\" because the premise provides additional specific details that are consistent with the hypothesis. The hypothesis states that a man is running and the premise states that a man is running, the premise also provides specific details about the man's clothing and the building he is running past, which are all consistent with the hypothesis. Therefore, the premise entails or supports the hypothesis.",




"sample 48: explain why the relation is entailment between hypothesis: the man is indoors. and premise: a man is sitting in the middle of a room in an art gallery, and is putting some frames together, preparing a piece of art for hanging.</s>",
"The relation is entailment between the hypothesis \"the man is indoors\" and the premise \"a man is sitting in the middle of a room in an art gallery, and is putting some frames together, preparing a piece of art for hanging\" because the premise provides additional information that confirms that the man is indeed indoors. The hypothesis says that the man is indoors, and the premise provides evidence that supports this by describing the location of the man as being in a room in an art gallery, which is typically an indoor space.",




"sample 49: explain why the relation is entailment between hypothesis: a man working hard. and premise: man in sweater stacks wooden crates next to pile of plastic crates.</s>",
"The relation is entailment because the premise provides additional information that supports the hypothesis. The hypothesis states that the man is working hard and the premise describes the man in specific detail, stacking wooden crates next to a pile of plastic crates. This action of stacking crates is an indication that the man is indeed working hard.",




"sample 50: explain why the relation is entailment between hypothesis: two humans are having a meal. and premise: these two people are eating.</s>",
"The relation is entailment between hypothesis: \"two humans are having a meal\" and premise: \"these two people are eating.\" because the premise \"these two people are eating\" is a specific instance of the more general statement in the hypothesis \"two humans are having a meal.\" The premise provides additional information that the two individuals in question are indeed people and that they are eating, which confirms the hypothesis.",




"sample 51: explain why the relation is neutral between hypothesis: the woman is walking back to her car after shopping. and premise: a woman wearing a blue dress and carrying a purple umbrella is walking in parking lot.</s>",
"The relation is neutral between the hypothesis \"the woman is walking back to her car after shopping\" and the premise \"a woman wearing a blue dress and carrying a purple umbrella is walking in a parking lot\" because the premise provides some information about the woman's appearance and location, but does not confirm or deny the specific action of her returning to her car after shopping. The premise does not provide enough information to confirm or contradict the hypothesis.",




"sample 52: explain why the relation is neutral between hypothesis: a man wearing a coat stands on a hill. and premise: a man wearing a yellow coat stands at the peak of a mountain.</s>",
"The relation between the hypothesis \"a man wearing a coat stands on a hill\" and the premise \"a man wearing a yellow coat stands at the peak of a mountain\" is neutral. This is because the hypothesis provides information about the man wearing a coat and standing on a hill, while the premise provides information about the man wearing a yellow coat and standing at the peak of a mountain. Both the hypothesis and the premise contain information about a man wearing a coat and standing on a hill, but the premise also includes additional information about the man's coat being yellow and him standing at the peak of a mountain. The additional information in the premise doesn't contradict or entail the information in the hypothesis, it's just supplementary information.",




"sample 53: explain why the relation is entailment between hypothesis: a boy in an ironic shirt cries. and premise: a young baby boy crying while wearing a shirt that says \"i am the boss.\"</s>",
"The relation is entailment because the premise provides additional information that the boy is crying, and also that he is wearing an ironic shirt. The hypothesis states that the boy is crying and the premise confirms this by stating that the boy is crying, while also providing additional information about the shirt he is wearing. Therefore, the premise logically follows from the hypothesis.",




"sample 54: explain why the relation is neutral between hypothesis: three people play gin rummy. and premise: two men and a woman playing cards.</s>",
"The relation is neutral between the hypothesis \"three people play gin rummy\" and the premise \"two men and a woman playing cards\" because while the premise states that there are two men and a woman playing cards, it does not specify the game they are playing. Therefore, it is possible that they could be playing gin rummy, but it is not confirmed.",




"sample 55: explain why the relation is neutral between hypothesis: the girl's costume is too short. and premise: a dancer performing with her panties showing.</s>",
"The relation is neutral between the hypothesis \"the girl's costume is too short\" and the premise \"a dancer performing with her panties showing\" because the hypothesis states a specific condition about the girl's costume (i.e., that it is too short), while the premise describes a general situation (i.e., a dancer performing) without specifying anything about the costume. It is possible that the dancer in the premise is the girl mentioned in the hypothesis, but without more information it is not clear if that is the case. Therefore, the two statements do not directly support or contradict each other.",




"sample 56: explain why the relation is contradiction between hypothesis: two boys ride regular bicycles during a race. and premise: two boys with unicycles stand outside of a street shop.</s>",
"The relation is contradiction between the hypothesis \"two boys ride regular bicycles during a race\" and the premise \"two boys with unicycles stand outside of a street shop\" because the hypothesis states that the boys are riding regular bicycles, while the premise states that they are riding unicycles. These two statements cannot both be true at the same time, as a regular bicycle and a unicycle are two different types of vehicles and therefore are mutually exclusive. Therefore, the relation is contradiction.",




"sample 57: explain why the relation is entailment between hypothesis: the two women are riding a cart outside. and premise: two asia women are riding on a cart as a man pulls them through the street.</s>",
"The relation is entailment between the hypothesis \"the two women are riding a cart outside\" and the premise \"two asia women are riding on a cart as a man pulls them through the street\" because the premise provides specific details that support and expand upon the information in the hypothesis. The hypothesis states that the women are riding a cart, and the premise provides additional information that they are Asian women, and that they are being pulled through the street by a man. This additional information supports and adds to the information provided in the hypothesis, making the premise an entailment of the hypothesis.",




"sample 58: explain why the relation is contradiction between hypothesis: boys are singing in the public. and premise: two male and six female dancers perform on stage.</s>",
"The relation is contradiction between hypothesis: boys are singing in the public. and premise: two male and six female dancers perform on stage, because the hypothesis states that boys are singing, while the premise states that two males and six females are dancing. These two statements cannot both be true at the same time, therefore they contradict each other.",




"sample 59: explain why the relation is contradiction between hypothesis: the kids are frowning and premise: children smiling and waving at camera</s>",
"The relation is contradiction because the hypothesis states that the kids are frowning, which is the opposite of the premise that states the children are smiling and waving at the camera. The two statements are mutually exclusive and cannot both be true at the same time.",




"sample 60: explain why the relation is neutral between hypothesis: three men in black shirts are at a table. and premise: three men, one of them standing, are at a table in a dark room.</s>",
"The relation between the hypothesis \"three men in black shirts are at a table\" and the premise \"three men, one of them standing, are at a table in a dark room\" is neutral because the premise does not provide enough information to confirm or contradict the hypothesis. The premise states that there are three men at a table, but it does not specify their clothing or the lighting conditions, so it is not possible to determine whether they match the description in the hypothesis.",





"sample 61: explain why the relation is contradiction between hypothesis: a person is at a diner, ordering an omelette. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation is a contradiction because the premise describes a person engaging in an activity that is not consistent with the setting described in the hypothesis. In the hypothesis, the person is at a diner ordering an omelette, which implies they are in an indoor, urban setting. However, in the premise, the person is on a horse jumping over a broken down airplane, which implies they are in a rural or wilderness setting, not a diner. These two settings are not compatible and therefore the hypothesis and premise contradict each other.",





"sample 62: explain why the relation is contradiction between hypothesis: a cart is sinking in the ocean and premise: a donkey pulled cart is carrying produce down a street.</s>",
"The relation is a contradiction between the hypothesis and the premise because the hypothesis states that a cart is sinking in the ocean, which implies that it is in water and unable to be used. However, the premise states that a donkey pulled cart is carrying produce down a street, which implies that the cart is on land and being used for transportation. These two statements cannot both be true at the same time and thus, the relation between the hypothesis and the premise is a contradiction.",




"sample 63: explain why the relation is entailment between hypothesis: a basketball game is being played. and premise: a group of basketball players jump in the air for the ball.</s>",
"The relation is entailment because the premise \"a group of basketball players jump in the air for the ball\" implies that a basketball game is being played. The action described in the premise, the players jumping for the ball, is a specific aspect of a basketball game and therefore the premise supports the hypothesis \"a basketball game is being played.\"",




"sample 64: explain why the relation is neutral between hypothesis: a person is wearing his church clothes as he lays on the ground. and premise: a person wearing a blue shirt, gray pants, and black shoes is laying on the ground next to a crutch.</s>",
"The relation is neutral because the premise does not specify that the person is wearing \"church clothes\" as the hypothesis states. The premise provides a description of the person's clothing, but it does not provide enough information to confirm or contradict the hypothesis that the person is wearing \"church clothes.\"",




"sample 65: explain why the relation is entailment between hypothesis: a large brown dog is standing oustide. and premise: a large brown dog is sticking his face in the sprinkler.</s>",
"The relation is entailment because the premise provides additional information about the large brown dog in the hypothesis, specifically that it is standing outside and sticking its face in a sprinkler, which is an action that a dog standing outside may do. The premise does not contradict the hypothesis, but rather adds to it.",




"sample 66: explain why the relation is neutral between hypothesis: women are standing on the sidewalk. and premise: two women conversing on a sidewalk.</s>",
"The relation is neutral between the hypothesis \"women are standing on the sidewalk\" and the premise \"two women conversing on a sidewalk\" because the hypothesis states that the women are standing, while the premise does not specify whether they are standing or sitting. Both statements are true, but the premise doesn't give additional information to the hypothesis.",




"sample 67: explain why the relation is entailment between hypothesis: a man is being carried. and premise: the long-haired man is balancing on a drum that is being carried by men in white shirts.</s>",
"The relation is entailment because the premise \"the long-haired man is balancing on a drum that is being carried by men in white shirts\" provides more information and context about the hypothesis \"a man is being carried.\" The additional information in the premise confirms that the man is indeed being carried by other men and adds detail to the image of the scene.",




"sample 68: explain why the relation is neutral between hypothesis: the man is very thirsty. and premise: man sitting by himself on a pier with a bottle of water.</s>",
"The relation is neutral because the premise provides information about a man sitting with a bottle of water, but it does not provide any information about his level of thirst. The hypothesis states that the man is very thirsty, but this information is not confirmed by the premise.",




"sample 69: explain why the relation is neutral between hypothesis: the 3 people are trying to figure out what the sign on the door means and are a little worried that they may be in the wrong room. and premise: two women and a man in a room that has a sign reading \"artisti\" on its door.</s>",
"The relation is neutral because while the hypothesis suggests that the three people are trying to figure out what the sign on the door means and are a little worried that they may be in the wrong room, the premise simply states that two women and a man are in a room that has a sign reading \"artisti\" on its door. The premise does not provide any information about the people's actions or emotions, so it cannot confirm or deny the hypothesis.",




"sample 70: explain why the relation is contradiction between hypothesis: man sets in cage and plays with rabbits. and premise: woman sets in cage and plays with rabbits.</s>",
"The relation is contradiction because the hypothesis states that a man is in the cage and playing with rabbits, while the premise states that a woman is in the cage and playing with rabbits. These statements cannot both be true at the same time, as they describe different genders of the person in the cage. This is a clear contradiction.",





"sample 71: explain why the relation is entailment between hypothesis: at least two people are in the pool. and premise: the little boy is in the swimming pool with an adult woman.</s>",
"The relation is entailment because the premise states that there is at least one adult woman and one little boy in the pool, which meets the minimum requirement stated in the hypothesis of \"at least two people\" being in the pool. The premise provides additional information about the specific individuals in the pool, but it does not contradict the statement in the hypothesis.",




"sample 72: explain why the relation is entailment between hypothesis: men and women are working in a neighborhood. and premise: a group of roughly 14 men and women works at a construction site in a residential area.</s>",
"The relation is entailment because the hypothesis states that \"men and women are working in a neighborhood\" and the premise states that \"a group of roughly 14 men and women works at a construction site in a residential area.\" The premise provides specific details that support the general statement made in the hypothesis, and therefore the premise entails the hypothesis. In other words, if the premise is true, the hypothesis is also true.",




"sample 73: explain why the relation is contradiction between hypothesis: the woman is cooking dinner. and premise: a woman is bending over a colorfully striped, metal couch on a sidewalk that has a bucket sitting atop it.</s>",
"The relation is contradiction because the hypothesis suggests the woman is engaged in the activity of cooking dinner, which typically takes place indoors and involves the use of a kitchen, whereas the premise describes the woman as being outdoors on a sidewalk, in proximity to a couch and a bucket. These two descriptions are not consistent with each other, and therefore the hypothesis and premise contradict each other.",




"sample 74: explain why the relation is neutral between hypothesis: two vehicles were parked next to a toyota. and premise: a black truck and a red car parked on the street in front of a grocery store painted red.</s>",
"The relation is neutral between the hypothesis \"two vehicles were parked next to a toyota\" and the premise \"a black truck and a red car parked on the street in front of a grocery store painted red\" because while the premise provides information about two specific vehicles (a black truck and a red car) being parked on the street, it does not specify the location of the vehicles in relation to one another. The hypothesis implies that there is a Toyota parked next to the two vehicles, but this information is not provided in the premise. Therefore, the premise does not confirm or deny the claim made in the hypothesis.",




"sample 75: explain why the relation is neutral between hypothesis: she is teaching her son to read. and premise: a mother is sitting and reading with her son.</s>",
"The relation is neutral between the hypothesis \"she is teaching her son to read\" and the premise \"a mother is sitting and reading with her son\" because the premise provides no information on whether the mother is actively teaching the son to read or if the son is reading independently with the help of the mother. The premise simply states that the mother is sitting and reading with her son, which could be interpreted as teaching, but it is not explicitly stated.",




"sample 76: explain why the relation is entailment between hypothesis: two people are by a tree. and premise: two kids stand under an area carved in a large tree.</s>",
"The relation is entailment between the hypothesis \"two people are by a tree\" and the premise \"two kids stand under an area carved in a large tree\" because the premise provides additional information about the location of the two people (i.e., that they are standing under an area carved in a large tree) and confirms that the hypothesis is true. The premise also provides information about the two people being kids, which is not mentioned in the hypothesis but it is entailed by it.",




"sample 77: explain why the relation is neutral between hypothesis: it is raining and premise: a young boy wearing a bright yellow raincoat, jean shorts and sandals squats down as he looks at an older boy and girl who are standing next to him and looking down some railroad tracks.</s>",
"The relation is neutral because the premise describes a specific scene with specific details, while the hypothesis simply states that it is raining. The premise does not confirm or deny the hypothesis that it is raining, it simply describes a scene that could be happening in the rain. While the boy is wearing a yellow raincoat, it doesn't confirm that it is raining, it could be worn for protection. Thus the premise does not provide any information that would contradict the hypothesis.",




"sample 78: explain why the relation is entailment between hypothesis: the people are going upstairs. and premise: several people walk up a staircase.</s>",
"The relation between the hypothesis \"the people are going upstairs\" and the premise \"several people walk up a staircase\" is entailment, because the premise provides additional information that confirms the hypothesis. The hypothesis states that the people are going upstairs, and the premise shows that they are physically walking up a staircase, which confirms that they are indeed going upstairs.",




"sample 79: explain why the relation is neutral between hypothesis: an older man drinks his juice as he waits for his daughter to get off work. and premise: an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.</s>",
"The relation is neutral between the hypothesis and premise because while the hypothesis states that the older man is drinking his juice and waiting for his daughter to get off work, the premise only describes the older man sitting with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background. The premise does not provide any information about the older man's daughter or why he is waiting. Therefore, the hypothesis and premise are not mutually exclusive or mutually dependent and do not contradict or entail each other.",




"sample 80: explain why the relation is contradiction between hypothesis: the man is standing. and premise: a man sitting on a curb in a city park.</s>",
"The relation is a contradiction because the hypothesis states that the man is standing, while the premise states that the man is sitting on a curb. These two statements cannot both be true at the same time, as standing and sitting are mutually exclusive actions. Therefore, the hypothesis and the premise are in direct opposition to each other.",




"sample 81: explain why the relation is entailment between hypothesis: a man is taking a run on the track. and premise: a man running on a track.</s>",
"The relation between the hypothesis \"a man is taking a run on the track\" and the premise \"a man running on a track\" is entailment because the premise provides additional information that confirms the hypothesis. The hypothesis states that a man is taking a run, which implies that he is moving, and the premise confirms that the man is indeed running, which is a specific type of movement. Therefore, the premise provides evidence that supports the hypothesis, making them consistent with each other.",




"sample 82: explain why the relation is neutral between hypothesis: a person is resting. and premise: a man crouching down on some rocks in the ocean.</s>",
"The relation is neutral because the premise describes a specific action (crouching on rocks) that is not necessarily indicative of resting. The hypothesis states that the person is resting, but the premise does not provide any information to confirm or deny that statement.",




"sample 83: explain why the relation is entailment between hypothesis: a woman stands at a podium. and premise: a middle-aged woman stands at a green podium with the name \"sva theatre\" written on it.</s>",
"The relation is entailment between the hypothesis \"a woman stands at a podium\" and the premise \"a middle-aged woman stands at a green podium with the name \"sva theatre\" written on it\" because the premise provides additional information that supports the hypothesis. The hypothesis states that a woman is standing at a podium, and the premise states that a middle-aged woman is standing at a green podium with a specific name, \"sva theatre\" written on it, which confirms the hypothesis that a woman is standing at a podium. Therefore, the premise entails the hypothesis.",




"sample 84: explain why the relation is contradiction between hypothesis: a man wearing a cowboy hat swings a baseball bat. and premise: a man wearing only a cowboy hat, cowboy boots, and underwear talks to a woman on a city street.</s>",
"The relation is contradiction between the hypothesis \"a man wearing a cowboy hat swings a baseball bat\" and the premise \"a man wearing only a cowboy hat, cowboy boots, and underwear talks to a woman on a city street\" because the premise states that the man is only wearing a cowboy hat, cowboy boots, and underwear and is talking to a woman on a city street, while the hypothesis states that the man is wearing a cowboy hat and swinging a baseball bat, which are two completely different and mutually exclusive actions and scenarios. Therefore, the hypothesis and the premise are contradictory.",




"sample 85: explain why the relation is contradiction between hypothesis: the man is lying in bed. and premise: a man and woman dressed in old fashion clothing sit outside at a table.</s>",
"The relation is a contradiction because the hypothesis states that the man is lying in bed, which is an indoor and sedentary activity, while the premise describes the man and woman sitting outside at a table, which is an outdoor and active activity. These two descriptions are mutually exclusive and cannot both be true at the same time, hence the contradiction.",




"sample 86: explain why the relation is entailment between hypothesis: store has people standing around it's exterior. and premise: asian people are standing outside a store named concepts.</s>",
"The relation between the hypothesis \"store has people standing around it's exterior\" and the premise \"asian people are standing outside a store named concepts\" is entailment because the premise provides evidence that supports the hypothesis. The premise specifically states that there are people standing outside a store, and the store is named \"concepts\", which confirms that there is a store with people standing around its exterior. Therefore, the premise entails the hypothesis, meaning that the hypothesis is logically supported by the premise.",





"sample 87: explain why the relation is contradiction between hypothesis: there are people running and premise: couple walking passed a clock tower.</s>",
"The relation is a contradiction between the hypothesis \"there are people running\" and the premise \"couple walking passed a clock tower\" because the hypothesis states that people are running, while the premise states that people are walking. These two statements are incompatible and cannot both be true at the same time. The fact that people are running implies that they are moving quickly and with purpose, while the fact that people are walking implies a slower and more leisurely pace. The contradiction is that the two statements give different information about the activity of the people.",




"sample 88: explain why the relation is contradiction between hypothesis: the men are watching a movie in public theatres in different states and premise: two men handle a large telescope while one is looking through.</s>",
"The relation is contradiction because the hypothesis states that the men are watching a movie in public theatres in different states, while the premise states that two men are handling a large telescope and one is looking through it. These two statements are mutually exclusive, as it is unlikely that the men would be able to watch a movie in public theatres in different states while also handling a telescope at the same time. Therefore, the hypothesis and premise are contradictory.",




"sample 89: explain why the relation is entailment between hypothesis: people are playing. and premise: a group of native adults and children are sitting in a circle playing a game.</s>",
"The relation between the hypothesis \"people are playing\" and the premise \"a group of native adults and children are sitting in a circle playing a game\" is entailment because the premise provides specific information that supports the general idea presented in the hypothesis. The premise describes a specific instance of people playing, which is what the hypothesis states is happening. The premise provides additional details, such as that the people are native and that they are playing a game, but these details do not contradict or negate the general idea that people are playing, as stated in the hypothesis. Therefore, the premise entails the hypothesis.",




"sample 90: explain why the relation is neutral between hypothesis: the men are related and premise: a man sitting with an elderly homeless person in a park.</s>",
"The relation is neutral because the premise does not provide any information about the relationship between the two men. It only states that they are sitting together in a park. The hypothesis, on the other hand, states that they are related, but it is not specified how they are related, and there is no information from the premise that can confirm or deny this claim. Therefore, it is not possible to determine if the hypothesis is true or false based on the premise.",




"sample 91: explain why the relation is neutral between hypothesis: a man is competing on a bike. and premise: a male doing a trick in the air on his bike.</s>",
"The relation is neutral between the hypothesis \"a man is competing on a bike\" and the premise \"a male doing a trick in the air on his bike\" because the premise does not indicate whether the man is competing or not, it only describes an action of a man on a bike. The premise does not provide any information about a competition or a race, so it does not confirm or contradict the hypothesis.",




"sample 92: explain why the relation is neutral between hypothesis: some artists are painting a wall. and premise: four young women are painting in countries of a map that is painted on a wall.</s>",
"The relation is neutral between the hypothesis \"some artists are painting a wall\" and the premise \"four young women are painting in countries of a map that is painted on a wall\" because the premise describes a specific situation, but the hypothesis is more general and does not specify that the artists are women or that they are painting a map on a wall. The hypothesis is compatible with the premise, but the premise does not provide any additional information or context to confirm or deny the hypothesis.",




"sample 93: explain why the relation is contradiction between hypothesis: a woman takes a bath. and premise: three people serving food outside somewhere.</s>",
"The relation is contradiction between the hypothesis \"a woman takes a bath\" and the premise \"three people serving food outside somewhere\" because the two statements describe two completely different and unrelated actions that cannot coexist at the same time. The hypothesis describes an individual doing a solitary activity in a private setting, while the premise describes a group of individuals performing an interactive activity in a public setting. The two statements are not compatible with each other and therefore contradict each other.",




"sample 94: explain why the relation is neutral between hypothesis: the woman is trying to make a living. and premise: an asian woman is selling colorful fabrics.</s>",
"The relation is neutral between the hypothesis \"the woman is trying to make a living\" and the premise \"an asian woman is selling colorful fabrics\" because while the premise describes an action that may be associated with someone trying to make a living, it does not directly confirm or refute the hypothesis. The premise simply provides a visual image of the woman selling fabrics, without any additional context or information about her motivations or circumstances.",





"sample 95: explain why the relation is contradiction between hypothesis: the boy skates down the sidewalk. and premise: a boy is jumping on skateboard in the middle of a red bridge.</s>",
"The relation is a contradiction between the hypothesis \"the boy skates down the sidewalk\" and the premise \"a boy is jumping on a skateboard in the middle of a red bridge\" because the hypothesis suggests that the boy is moving on the sidewalk, while the premise suggests that the boy is on a skateboard in the middle of a red bridge. These two statements cannot both be true at the same time, as the sidewalk and red bridge are different locations and the boy cannot be in both places simultaneously. Additionally, the action described in the hypothesis \"skating down\" is different from the action described in the premise \"jumping on\". Therefore, the two statements contradict each other.",
]

ral_list = []
inp_list = []
ral_inp_dict = {}
for kk in range(96):
    inp_list.append(chatgpt_rationales[2*kk][chatgpt_rationales[2*kk].index("explain why the relation is"):])
    ral_list.append(chatgpt_rationales[2*kk+1])
    ral_inp_dict[chatgpt_rationales[2*kk][chatgpt_rationales[2*kk].index("explain why the relation is"):]] = chatgpt_rationales[2*kk+1]
    
    

In [19]:
new_refine_train_data = []
for item in refine_train_data:
    premise = item["premise"]
    hypothesis = item["hypothesis"]
    answer = wt5_esnli_label_mapping[item["label"]]
    input_string = f'explain why the relation is {answer} between hypothesis: {hypothesis.lower()} and premise: {premise.lower()}</s>'  
    print(input_string)
    if input_string in ral_inp_dict.keys():
        item['our_explanation'] = ral_inp_dict[input_string]
        new_refine_train_data.append(item)

new_refine_train_data
    

explain why the relation is neutral between hypothesis: a person is training his horse for a competition. and premise: a person on a horse jumps over a broken down airplane.</s>
explain why the relation is contradiction between hypothesis: a person is at a diner, ordering an omelette. and premise: a person on a horse jumps over a broken down airplane.</s>
explain why the relation is entailment between hypothesis: a person is outdoors, on a horse. and premise: a person on a horse jumps over a broken down airplane.</s>
explain why the relation is neutral between hypothesis: they are smiling at their parents and premise: children smiling and waving at camera</s>
explain why the relation is entailment between hypothesis: there are children present and premise: children smiling and waving at camera</s>
explain why the relation is contradiction between hypothesis: the kids are frowning and premise: children smiling and waving at camera</s>
explain why the relation is contradiction between hy

[{'premise': 'A person on a horse jumps over a broken down airplane.',
  'hypothesis': 'A person is training his horse for a competition.',
  'label': 1,
  'explanation_1': 'the person is not necessarily training his horse',
  'explanation_2': '',
  'explanation_3': '',
  'our_explanation': 'The relation between the hypothesis "a person is training his horse for a competition" and the premise "a person on a horse jumps over a broken down airplane" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that a person on a horse jumps over a broken down airplane, but it does not specify what the person and horse are doing. It could be that they are training for a competition, but it could also be that they\'re just doing some exercise or practicing jumping or obstacle course. The premise does not indicate if the person is training the horse for a competition or not. Therefore, the premise does not provide enough information

In [20]:
new_refine_train_data

[{'premise': 'A person on a horse jumps over a broken down airplane.',
  'hypothesis': 'A person is training his horse for a competition.',
  'label': 1,
  'explanation_1': 'the person is not necessarily training his horse',
  'explanation_2': '',
  'explanation_3': '',
  'our_explanation': 'The relation between the hypothesis "a person is training his horse for a competition" and the premise "a person on a horse jumps over a broken down airplane" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that a person on a horse jumps over a broken down airplane, but it does not specify what the person and horse are doing. It could be that they are training for a competition, but it could also be that they\'re just doing some exercise or practicing jumping or obstacle course. The premise does not indicate if the person is training the horse for a competition or not. Therefore, the premise does not provide enough information

In [21]:
ral_inp_dict.keys()

dict_keys(['explain why the relation is neutral between hypothesis: they are smiling at their parents and premise: children smiling and waving at camera</s>', 'explain why the relation is contradiction between hypothesis: two people are playing cribbage. and premise: two men in safety vests walk down the road.</s>', 'explain why the relation is neutral between hypothesis: the tourists are getting a gondola ride. and premise: the tourists are out on the river in italy, the man in the black and white striped shirt will tell them about the seine.</s>', 'explain why the relation is neutral between hypothesis: a person is training his horse for a competition. and premise: a person on a horse jumps over a broken down airplane.</s>', 'explain why the relation is contradiction between hypothesis: the man is in a desert. and premise: an man in the north pole walking on ice.</s>', 'explain why the relation is entailment between hypothesis: a woman ropes an animal. and premise: a woman on the bac

In [22]:
def list2dict(refine_data):
    refine_data_dict={}
    for key in refine_data[0].keys():
        refine_data_dict[key]=[x[key] for x in refine_data]
    return refine_data_dict

new_refine_train_data_dict = list2dict(new_refine_train_data)
our_data_splits['train'] = datasets.Dataset.from_dict(new_refine_train_data_dict).shuffle().select(range(72))

refine_dev_data_dict = list2dict(refine_dev_data)
our_data_splits['dev'] = datasets.Dataset.from_dict(refine_dev_data_dict)

refine_test_data_dict = list2dict(refine_test_data)
our_data_splits['test'] = datasets.Dataset.from_dict(refine_test_data_dict)



In [23]:
import datasets
class SequenceCollator:
    def __init__(self, pad_token):
        # self.pad_token_mapping = {
        #     "lm_labels": -100,
        #     "attention_mask": 0,
        #     "decoder_attention_mask": 0,
        #     "input_ids": pad_token,
        # }
        # self.columns = [
        #     "input_ids",
        #     "attention_mask",
        #     "lm_labels",
        #     "decoder_attention_mask",
        # ]
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }
        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def collate_batch(self, examples):

        # batch inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch
    
    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch
# dataset = datasets.load_dataset(data_args.task_name, data_args.version_name)

In [24]:
# seq_collector = SequenceCollator(0)
# train_ds = seq_collector.__call__(dataset['train'])
# train_ds
# dataset['train'][0].keys()
for split in ['train','dev','test']:
    our_data_splits[split] = our_data_splits[split].map(
            lambda x: format_instance(
                x,
                tokenizer,
                data_args.explanation_sep,
                datasource=data_args.task_name,
                io_format=data_args.io_format
            ),
            batched=False,
            load_from_cache_file=False,
        )

  0%|          | 0/72 [00:00<?, ?ex/s]

  0%|          | 0/640 [00:00<?, ?ex/s]

  0%|          | 0/131 [00:00<?, ?ex/s]

In [25]:
for kk, da in enumerate(our_data_splits['train']):
    input_string = tokenizer.decode(da['input_ids'])
    print(f'{input_string}')
    print(tokenizer.decode(da['labels']))

explain why the relation is contradiction between hypothesis: the horses are in their stalls in the barn. and premise: several horses are running a race on a muddy track.</s>
<extra_id_0> The relation is contradiction because the hypothesis states that the horses are in their stalls in the barn, which implies that they are not moving or engaging in any activity. The premise, on the other hand, states that several horses are running a race on a muddy track, which is an activity that would not be possible if the horses were in their stalls in the barn. These two statements cannot both be true at the same time, and therefore, they contradict each other.<extra_id_1> </s>
explain why the relation is contradiction between hypothesis: two people are playing cribbage. and premise: two men in safety vests walk down the road.</s>
<extra_id_0> The relation between the hypothesis "two people are playing cribbage" and the premise "two men in safety vests walk down the road" is a contradiction becau

explain why the relation is contradiction between hypothesis: the kids are frowning and premise: children smiling and waving at camera</s>
<extra_id_0> The relation is contradiction because the hypothesis states that the kids are frowning, which is the opposite of the premise that states the children are smiling and waving at the camera. The two statements are mutually exclusive and cannot both be true at the same time.<extra_id_1> </s>
explain why the relation is neutral between hypothesis: two vehicles were parked next to a toyota. and premise: a black truck and a red car parked on the street in front of a grocery store painted red.</s>
<extra_id_0> The relation is neutral between the hypothesis "two vehicles were parked next to a toyota" and the premise "a black truck and a red car parked on the street in front of a grocery store painted red" because while the premise provides information about two specific vehicles (a black truck and a red car) being parked on the street, it does n

In [26]:
chatgpt_rationales = ["sample 0: explain why the relation is neutral between hypothesis: they are smiling at their parents and premise: children smiling and waving at camera</s>",
"The relation between the hypothesis \"they are smiling at their parents\" and the premise \"children smiling and waving at camera\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that children are smiling and waving at the camera, but it does not specify who they are smiling and waving at, so it is possible that they are smiling and waving at their parents, but it is also possible that they are smiling and waving at someone else. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 1: explain why the relation is contradiction between hypothesis: two people are playing cribbage. and premise: two men in safety vests walk down the road.</s>",
"The relation between the hypothesis \"two people are playing cribbage\" and the premise \"two men in safety vests walk down the road\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that two people are playing cribbage, which implies they are in a stationary location, most likely indoors, and are engaged in a game. On the other hand, the premise states that two men are walking down the road, which implies that they are in motion, and most likely outdoors. These two statements cannot both be true at the same time as the people described in the hypothesis are not doing the same activity as the men described in the premise. Therefore, the relation between these two statements is a contradiction.",





"sample 2: explain why the relation is neutral between hypothesis: the tourists are getting a gondola ride. and premise: the tourists are out on the river in italy, the man in the black and white striped shirt will tell them about the seine.</s>",
"The relation between the hypothesis \"the tourists are getting a gondola ride\" and the premise \"the tourists are out on the river in Italy, the man in the black and white striped shirt will tell them about the Seine\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that the tourists are out on the river in Italy and that a man in a black and white striped shirt will tell them about the Seine, but it does not specify what the tourists are doing or where they are exactly. The premise does not specify if they are in a gondola or not, so it is possible that they are getting a gondola ride, but it is also possible that they are doing something else. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 3: explain why the relation is neutral between hypothesis: a person is training his horse for a competition. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation between the hypothesis \"a person is training his horse for a competition\" and the premise \"a person on a horse jumps over a broken down airplane\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The premise states that a person on a horse jumps over a broken down airplane, but it does not specify what the person and horse are doing. It could be that they are training for a competition, but it could also be that they're just doing some exercise or practicing jumping or obstacle course. The premise does not indicate if the person is training the horse for a competition or not. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",



"sample 4: explain why the relation is contradiction between hypothesis: the man is in a desert. and premise: an man in the north pole walking on ice.</s>",
"The relation between the hypothesis \"the man is in a desert\" and the premise \"a man in the North Pole walking on ice\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that the man is in a desert, which is a hot and dry area with little vegetation, while the premise states that the man is in the North Pole, which is a cold and icy area with no vegetation. These two statements cannot both be true at the same time as they describe two completely different environments and conditions. Therefore, the relation between these two statements is a contradiction.",




"sample 5: explain why the relation is entailment between hypothesis: a woman ropes an animal. and premise: a woman on the back of a horse ropes a cattle.</s>",
"The relation between the hypothesis \"a woman ropes an animal\" and the premise \"a woman on the back of a horse ropes a cattle\" is entailment because the premise confirms the hypothesis and provides additional information. The hypothesis states that a woman ropes an animal, which is a general statement, and the premise provides specific information that confirms the general statement, that a woman on the back of a horse ropes a cattle. The premise specifies that the woman is on the back of a horse and she ropes a cattle, which is an animal, which is the general statement that the hypothesis makes. The premise provides additional information, the woman is on the back of a horse and the animal she ropes is a cattle which is specific and confirms the general statement of the hypothesis. Therefore, the relation between these two statements is entailment.",




"sample 6: explain why the relation is contradiction between hypothesis: sports aren't played they are worn and premise: a young lacrosse player is catching his breath.</s>",
"The relation between the hypothesis \"Sports aren't played they are worn\" and the premise \"A young lacrosse player is catching his breath\" is a contradiction because the two statements cannot both be true at the same time. The hypothesis states that sports are not played, they are worn, which implies that they are not an activity but rather a clothing or gear. On the other hand, the premise states that a young lacrosse player is catching his breath, which implies that he is engaged in an activity that requires physical effort and exertion, that is playing lacrosse. These two statements cannot both be true at the same time as the lacrosse player is playing and not wearing it. Therefore, the relation between these two statements is a contradiction.",





"sample 7: explain why the relation is neutral between hypothesis: a surgeon performs open heart surgery. and premise: a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs.</s>",
"The relation between the hypothesis \"a surgeon performs open heart surgery\" and the premise \"a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that a surgeon performs open heart surgery, which is a specific type of surgery, and the premise provides information about a male surgeon wearing a red cap and teal scrubs performing surgery with two other men wearing teal scrubs. The premise does not specify what type of surgery is being performed, and it only describes the attire and appearance of the surgeons and the fact that they are performing surgery. It does not indicate whether the surgery is open heart surgery or not. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",





"sample 8: explain why the relation is neutral between hypothesis: a man awaits the results of his hiv test. and premise: a man in a blue long-sleeved jacket crying out of fear.</s>",
"The relation between the hypothesis \"a man awaits the results of his HIV test\" and the premise \"a man in a blue long-sleeved jacket crying out of fear\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that a man awaits the results of his HIV test, which implies that the man is waiting for the results of a medical test for HIV. The premise states that a man in a blue long-sleeved jacket crying out of fear, which implies that the man is experiencing fear and is crying. The premise does not indicate whether the man is waiting for the results of an HIV test or not, and it only describes the attire and appearance of the man and his emotional state. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 9: explain why the relation is neutral between hypothesis: the people are trying to get their ball back. and premise: people holding onto ropes while trying to get something through a window on the second floor.</s>",
"The relation between the hypothesis \"The people are trying to get their ball back\" and the premise \"people holding onto ropes while trying to get something through a window on the second floor\" is neutral because the premise does not provide enough information to confirm or deny the hypothesis. The hypothesis states that the people are trying to get their ball back, which implies they have lost a ball and they are trying to retrieve it. The premise describes people holding onto ropes while trying to get something through a window on the second floor, which implies that the people are performing some sort of activity involving ropes and a window on the second floor. The premise does not indicate whether they are trying to get a ball back or not, and it only describes the activity that people are doing and the context. Therefore, the premise does not provide enough information to confirm or deny the hypothesis.",




"sample 10: explain why the relation is entailment between hypothesis: he is happy. and premise: a boy in colorful pants plays guitar and laughs together with his friends.</s>",
"The relation between the hypothesis \"he is happy\" and the premise \"a boy in colorful pants plays guitar and laughs together with his friends\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"he is happy\" which implies that the person is in a positive emotional state. The premise describes a boy in colorful pants who is playing guitar and laughing together with his friends, which implies that the boy is enjoying himself and having a good time. Laughter and playing guitar with friends is a strong indication of happiness. Therefore, the premise confirms the hypothesis that \"he is happy\".",




"sample 11: explain why the relation is entailment between hypothesis: the people are outside. and premise: people are walking around a park area.</s>",
"The relation between the hypothesis \"the people are outside\" and the premise \"people are walking around a park area\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"the people are outside\", which implies that the people are in an open-air environment and not indoors. The premise describes people walking around a park area, which implies that the people are outside in a public park where people can walk and enjoy the nature. This confirms that the people are outside, as the park area is an open-air environment. Therefore, the premise confirms the hypothesis that \"the people are outside\".",




"sample 12: explain why the relation is entailment between hypothesis: two men are with animals. and premise: two men chase cattle on horses.</s>",
"The relation between the hypothesis \"two men are with animals\" and the premise \"two men chase cattle on horses\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"two men are with animals\", which implies that the two men are in the presence of or interacting with animals. The premise describes two men chasing cattle on horses, which implies that the two men are with animals as they are interacting with cattle, which are animals. Additionally, the fact that they are doing so on horses also confirms that they are with animals. Therefore, the premise confirms the hypothesis that \"two men are with animals\".",




"sample 13: explain why the relation is neutral between hypothesis: the two men play very well. and premise: two men are playing acoustic guitars.</s>",
"The relation between the hypothesis \"the two men play very well\" and the premise \"two men are playing acoustic guitars\" is neutral because the premise does not confirm or deny the hypothesis. The hypothesis states that \"the two men play very well,\" which implies that the two men have a high level of skill or proficiency in playing the guitars. The premise describes two men playing acoustic guitars, which does not confirm or deny the hypothesis. It is not specified whether the two men are playing well or not. It could be that they are playing well, poorly, or somewhere in between. The premise does not provide any information about the skill level of the men playing the guitars, making the relation neutral.",



"sample 14: explain why the relation is contradiction between hypothesis: an elderly woman bathes and premise: a child is laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone.</s>",
"The relation between the hypothesis \"an elderly woman bathes\" and the premise \"a child is laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone\" is contradiction because the premise denies the hypothesis. The hypothesis states that \"an elderly woman bathes,\" which implies that an elderly woman is in the process of cleaning herself with water. The premise describes a child laying in a bubble bath holding a yellow scrubbing brush up to his mouth as if singing into a microphone. This implies that the person in the bathtub is a child and not an elderly woman. This contradicts the hypothesis as the premise is describing a person that is not elderly and not a woman. Therefore, the premise denies the hypothesis that \"an elderly woman bathes\".",




"sample 15: explain why the relation is entailment between hypothesis: a person is outdoors, on a horse. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation between the hypothesis \"a person is outdoors, on a horse\" and the premise \"a person on a horse jumps over a broken down airplane\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"a person is outdoors, on a horse,\" which implies that the person is outside and mounted on a horse. The premise describes a person on a horse jumping over a broken down airplane, which confirms the hypothesis that the person is outside and mounted on a horse. The premise provides additional information about the person's location and activity, but it does not contradict the main idea that the person is outside and on a horse. Therefore, the premise confirms the hypothesis \"a person is outdoors, on a horse\".",




"sample 16: explain why the relation is entailment between hypothesis: a girl and a boy stick their tongues out. and premise: a girl with a little boy both rolling their tongue as they stick it out.</s>",
"The relation between the hypothesis \"a girl and a boy stick their tongues out\" and the premise \"a girl with a little boy both rolling their tongue as they stick it out\" is entailment because the premise confirms the hypothesis. The hypothesis states that \"a girl and a boy stick their tongues out\" which implies that at least one girl and one boy are sticking their tongues out. The premise describes a girl with a little boy both rolling their tongue as they stick it out, which confirms the hypothesis that at least one girl and one boy are sticking their tongues out. The premise provides additional information about the specific girl and boy and their action, but it does not contradict the main idea that at least one girl and one boy are sticking their tongues out. Therefore, the premise confirms the hypothesis \"a girl and a boy stick their tongues out\"",





"sample 17: explain why the relation is neutral between hypothesis: a boy was showing off for a girl. and premise: a young boy wearing a tank-top is climbing a tree.</s>",
"The relation between the hypothesis \"a boy was showing off for a girl\" and the premise \"a young boy wearing a tank-top is climbing a tree\" is neutral because the premise doesn't provide enough information to confirm or contradict the hypothesis. The hypothesis states that the boy was showing off for a girl, which implies that there is a girl present and the boy is doing something to impress her. The premise describes a young boy wearing a tank-top climbing a tree, which doesn't provide enough information to confirm or contradict the hypothesis. The premise doesn't tell us if there is a girl present or not and doesn't tell us if the boy is doing something to impress her or not. Therefore, the premise doesn't give enough information to confirm or contradict the hypothesis \"a boy was showing off for a girl\" thus the relation is neutral.",





"sample 18: explain why the relation is contradiction between hypothesis: three women are walking together outside. and premise: three women sitting and waiting for something on a bench.</s>",
"The relation between the hypothesis \"three women are walking together outside\" and the premise \"three women sitting and waiting for something on a bench\" is contradiction because the premise contradicts the hypothesis. The hypothesis states that the three women are walking together outside, which implies that they are moving around and not stationary. The premise describes three women sitting and waiting for something on a bench, which contradicts the idea that they are moving around and walking. The premise clearly shows that the women are sitting and not walking, therefore it contradicts the hypothesis that states that the three women are walking together outside, thus the relation is contradiction.",




"sample 19: explain why the relation is entailment between hypothesis: a woman singing and premise: a woman with brown hair wearing a red polka dot dress is singing into a microphone.</s>",
"The relation between the hypothesis \"a woman singing\" and the premise \"a woman with brown hair wearing a red polka dot dress is singing into a microphone\" is entailment because the premise provides more information that supports the hypothesis. The hypothesis states that a woman is singing, which means that she is producing sound with her voice. The premise describes a woman with brown hair wearing a red polka dot dress who is singing into a microphone, which confirms the hypothesis that a woman is singing. It also gives additional information such as the woman's appearance, clothing and the tool she's using to produce the sound (microphone) which supports the hypothesis that a woman is singing. Therefore, the premise entails the hypothesis, making the relation entailment.",




"sample 20: explain why the relation is neutral between hypothesis: a singer sings their most famous song. and premise: a singer entertaining the crowd.</s>",
"The relation is neutral between the hypothesis \"a singer sings their most famous song\" and the premise \"a singer entertaining the crowd\" because the premise does not specify what song the singer is singing, so it could be their most famous song, but it could also be a different song. Therefore, the premise does not support or contradict the hypothesis.",





"sample 21: explain why the relation is neutral between hypothesis: a guy is running from the police. and premise: a guy in white shirt and black pants is running.</s>",
"The relation is neutral between the hypothesis \"a guy is running from the police\" and the premise \"a guy in white shirt and black pants is running\" because the premise only states that a guy in a certain clothing is running, it doesn't specify the reason why he is running, so it could be from the police or for another reason, the premise does not support or contradict the hypothesis.",




"sample 22: explain why the relation is contradiction between hypothesis: the man is working on his car and premise: women walking down the street in jeans, black high heel boots, and a beige handbag.</s>",
"The relation is a contradiction because the hypothesis states that the man is working on his car, which suggests that he is engaged in a specific activity related to automobiles. However, the premise describes a group of women who are walking down the street, which is not related to the man working on a car and thus contradicts the hypothesis.",




"sample 23: explain why the relation is entailment between hypothesis: a man wields a tool and premise: a man is cutting a fish.</s>",
"The relation is entailment between the hypothesis \"a man wields a tool\" and the premise \"a man is cutting a fish\" because the premise provides specific information that supports the hypothesis. The hypothesis states that a man is using a tool and the premise specifies that the man is cutting a fish, which requires the use of a tool such as a knife. Therefore, the premise confirms that the man is indeed using a tool, and thus the two statements are logically consistent.",





"sample 24: explain why the relation is neutral between hypothesis: a band is playing a concert. and premise: this band is playing on a stage.</s>",
"The relation is neutral between the hypothesis \"a band is playing a concert\" and the premise \"this band is playing on a stage\" because both the hypothesis and the premise mention the band playing, but do not provide any information that would indicate whether the band is playing a concert or not. The premise only states that the band is playing on a stage, which could be part of a concert, but it does not confirm or deny the hypothesis that the band is playing a concert.",




"sample 25: explain why the relation is contradiction between hypothesis: the horses are becoming suspicious of my apples. and premise: there is a red truck behind the horses.</s>",
"The relation is contradiction because the premise (\"there is a red truck behind the horses\") does not match the hypothesis (\"the horses are becoming suspicious of my apples\"). The horse's behavior is described as being suspicious of apples, not having any relation with a red truck. The two statements are not related and cannot both be true at the same time.",




"sample 26: explain why the relation is neutral between hypothesis: two african children are friends. and premise: two african children standing by a hut with their faces painted.</s>",
"The relation is neutral between the hypothesis \"two african children are friends\" and the premise \"two african children standing by a hut with their faces painted\" because the premise does not provide any information about the relationship between the children. It only describes their physical appearance and location. The hypothesis states that the children are friends, but the premise does not confirm or deny this.",





"sample 27: explain why the relation is contradiction between hypothesis: a nurse is pushing a surgical cart and premise: a woman wearing gloves working at a meat counter.</s>",
"The relation is a contradiction because the hypothesis and premise describe two very different scenarios. In the hypothesis, a nurse is described as pushing a surgical cart in a hospital setting, while in the premise, a woman is described as working at a meat counter in a grocery store. These two images are not consistent with each other and therefore, the relation is a contradiction.",





"sample 28: explain why the relation is entailment between hypothesis: there are children present and premise: children smiling and waving at camera</s>",
"The relation between the hypothesis \"there are children present\" and the premise \"children smiling and waving at camera\" is entailment because the premise provides evidence that supports the hypothesis. The presence of children smiling and waving in the premise confirms that there are indeed children present, as stated in the hypothesis. The hypothesis is entailed by the premise because the premise is providing visual evidence of the presence of children.",





"sample 29: explain why the relation is contradiction between hypothesis: a girl is sitting at the kitchen table eating lunch. and premise: a girl, about 8 years old, is sitting in front of a television, delighted with a stuffed animal, while a boy watches.</s>",
"The relation is contradiction because the hypothesis describes the girl as sitting at the kitchen table eating lunch, while the premise describes the girl as sitting in front of a television, delighted with a stuffed animal, which would suggest that she is not eating lunch at the kitchen table. These two statements cannot both be true at the same time, making the relation a contradiction.",




"sample 30: explain why the relation is neutral between hypothesis: the girls is having fun at the park. and premise: the girl wears a purple outfit and swings upside down.</s>",
"The relation is neutral between the hypothesis \"the girl is having fun at the park\" and the premise \"the girl wears a purple outfit and swings upside down\" because the premise only describes the girl's appearance and actions, but it does not provide any information about the girl's emotional state or the context in which she is participating in those actions. The hypothesis suggests that the girl is having fun, but the premise does not confirm or deny this. It is possible that the girl is having fun at the park, but it is also possible that she is not.",




"sample 31: explain why the relation is neutral between hypothesis: the people are having a business meeting. and premise: an aerial view of a group of meeting talking to each other.</s>",
"The relation between the hypothesis \"the people are having a business meeting\" and the premise \"an aerial view of a group of people talking to each other\" is neutral because the premise provides general information about a group of people talking to each other, but it does not specify that they are having a business meeting. The premise does not necessarily support or contradict the hypothesis.",




"sample 32: explain why the relation is entailment between hypothesis: a woman and dog are together. and premise: dog on a red leash with a woman.</s>",
"The relation is entailment between the hypothesis \"a woman and dog are together\" and the premise \"dog on a red leash with a woman\" because the premise provides specific details that confirm the hypothesis. The premise states that there is a dog on a red leash with a woman, which implies that the woman and dog are together as stated in the hypothesis. The information in the premise supports and confirms the information in the hypothesis.",




"sample 33: explain why the relation is neutral between hypothesis: soccer players play in the world championship in front of a sell out audience. and premise: soccer players playing a game in front of a big crowd.</s>",
"The relation is neutral because the premise provides no information about the context of the soccer game, such as whether it is a world championship or whether the audience is a sell-out crowd, while the hypothesis does. The premise only states that soccer players are playing a game in front of a big crowd, which is not necessarily enough information to determine whether the game is a world championship or not.",





"sample 34: explain why the relation is entailment between hypothesis: a child is holding a picture and premise: child holds up a picture of mahmoud ahmadinejad while men converse behind him.</s>",
"The relation is entailment between the hypothesis \"a child is holding a picture\" and the premise \"child holds up a picture of mahmoud ahmadinejad while men converse behind him\" because the premise provides specific details about the picture the child is holding, and that it is of a specific person, Mahmoud Ahmadinejad. This additional information is entailed by the more general information in the hypothesis that the child is holding a picture.",




"sample 35: explain why the relation is contradiction between hypothesis: there are people dancing in the rain. and premise: a man and two women are browsing through a market that sells vases, jewelery and other items.</s>",
"The relation between the hypothesis \"there are people dancing in the rain\" and the premise \"a man and two women are browsing through a market that sells vases, jewelry, and other items\" is a contradiction because the two statements describe completely different actions and locations. The hypothesis implies that people are actively engaging in a specific activity (dancing) in a specific weather condition (rain), while the premise describes a completely different activity (browsing through a market) and location (indoors). These two statements cannot both be true at the same time, which is why the relation between them is a contradiction.",




"sample 36: explain why the relation is entailment between hypothesis: four boys are outside. and premise: four boys play in the mud.</s>",
"The relation is entailment between the hypothesis \"four boys are outside\" and the premise \"four boys play in the mud\" because the premise is a specific example of the situation described in the hypothesis. If it is known that there are four boys outside, it must be true that they are playing in the mud, as the premise states. The premise provides additional information that is entailed by the hypothesis.",




"sample 37: explain why the relation is contradiction between hypothesis: two motorcylists are driving up a mountain. and premise: a motorcyclist drives up the side of a green mountain while another bike remains stationary.</s>",
"The relation is a contradiction because the hypothesis states that two motorcyclists are driving up a mountain, but the premise states that only one motorcyclist is driving while the other bike remains stationary. The two statements cannot both be true at the same time, as only one motorcyclist is seen to be moving in the premise, while the hypothesis states that both are moving.",




"sample 38: explain why the relation is entailment between hypothesis: the boy does a skateboarding trick. and premise: a boy is jumping on skateboard in the middle of a red bridge.</s>",
"The relation is entailment because the premise \"a boy is jumping on Skateboard in the middle of a red bridge\" describes an action that is consistent with the hypothesis \"the boy does a skateboarding trick.\" The premise provides additional details and context to the hypothesis, but the basic action of the boy performing a skateboarding trick is still present in the premise.",




"sample 39: explain why the relation is contradiction between hypothesis: the horses are in their stalls in the barn. and premise: several horses are running a race on a muddy track.</s>",
"The relation is contradiction because the hypothesis states that the horses are in their stalls in the barn, which implies that they are not moving or engaging in any activity. The premise, on the other hand, states that several horses are running a race on a muddy track, which is an activity that would not be possible if the horses were in their stalls in the barn. These two statements cannot both be true at the same time, and therefore, they contradict each other.",




"sample 40: explain why the relation is neutral between hypothesis: an giant collected flowers and premise: an giant egged shaped collection of flowers.</s>",
"The relation is neutral between the hypothesis \"an giant collected flowers\" and the premise \"an giant egged shaped collection of flowers\" because the premise does not provide any information about whether the flowers were collected by a giant or not. The premise simply describes an object made of flowers that has an egg-shaped form, without providing any information about the action or context. Therefore, the premise does not support or contradict the hypothesis, and the relation is neutral.",




"sample 41: explain why the relation is neutral between hypothesis: women are competing in a table tennis tournament at the beach. and premise: two females in bikinis are playing table tennis at the beach while another female in a towel watches.</s>",
"The relation is neutral because the hypothesis and the premise both describe women competing in a table tennis tournament at the beach, but the premise also includes additional details such as what they are wearing. The additional details do not contradict or support the statement in the hypothesis, so the relation is neutral.",




"sample 42: explain why the relation is contradiction between hypothesis: the table was wood. and premise: two men sit at a glass table with papers on it.</s>",
"The relation is contradiction because the hypothesis states that the table is made of wood, while the premise states that the table is made of glass. These two statements cannot both be true at the same time, as a table cannot be made of both wood and glass. This is an example of a logical contradiction, as it is impossible for both statements to be true.",




"sample 43: explain why the relation is entailment between hypothesis: a bearded man plays guitar. and premise: a long bearded man plays a furry guitar.</s>",
"The relation is entailment between the hypothesis \"a bearded man plays guitar\" and the premise \"a long bearded man plays a furry guitar\" because the premise provides additional information about the man's beard and the guitar that he is playing, but it does not change the fact that the man is playing a guitar, which is stated in the hypothesis. The premise is consistent with the hypothesis and adds more details to it.",




"sample 44: explain why the relation is entailment between hypothesis: the woman in knee-highs and jean shorts is walking down the sidewalk smoking. and premise: a woman in checkered knee-highs and jean shorts walks down a city sidewalk street while smoking a cigarette.</s>",
"The relation is entailment because the premise provides additional information that supports the hypothesis. The specific details provided in the premise, such as the woman wearing knee-highs and jean shorts, and smoking a cigarette, are consistent with and support the general statement made in the hypothesis that a woman is walking down the sidewalk smoking.",




"sample 45: explain why the relation is contradiction between hypothesis: a boy flips a burger. and premise: an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.</s>",
"The relation is contradiction between the hypothesis \"a boy flips a burger\" and the premise \"an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background\" because the hypothesis states that a specific action (flipping a burger) is happening in a specific setting (presumably a kitchen or grill area), while the premise describes a completely different scene with no indication of a burger-related activity. These two statements cannot both be true at the same time.",




"sample 46: explain why the relation is neutral between hypothesis: two couples are preparing dinner together. and premise: two ladies are cooking & talking and two gentlemen are talking in the opposite site of the kitchen.</s>",
"The relation between the hypothesis and premise is neutral because the premise describes two couples engaging in separate activities in the kitchen, while the hypothesis states that they are preparing dinner together. The premise does not provide enough information to confirm or deny the hypothesis.",





"sample 47: explain why the relation is entailment between hypothesis: a man runs outside past a building. and premise: a man wearing blue jogging pants with 3 white stripes and a t-shirt is running past a building with graffiti on it.</s>",
"The relation is entailment between the hypothesis \"a man runs outside past a building\" and the premise \"a man wearing blue jogging pants with 3 white stripes and a t-shirt is running past a building with graffiti on it\" because the premise provides additional specific details that are consistent with the hypothesis. The hypothesis states that a man is running and the premise states that a man is running, the premise also provides specific details about the man's clothing and the building he is running past, which are all consistent with the hypothesis. Therefore, the premise entails or supports the hypothesis.",




"sample 48: explain why the relation is entailment between hypothesis: the man is indoors. and premise: a man is sitting in the middle of a room in an art gallery, and is putting some frames together, preparing a piece of art for hanging.</s>",
"The relation is entailment between the hypothesis \"the man is indoors\" and the premise \"a man is sitting in the middle of a room in an art gallery, and is putting some frames together, preparing a piece of art for hanging\" because the premise provides additional information that confirms that the man is indeed indoors. The hypothesis says that the man is indoors, and the premise provides evidence that supports this by describing the location of the man as being in a room in an art gallery, which is typically an indoor space.",




"sample 49: explain why the relation is entailment between hypothesis: a man working hard. and premise: man in sweater stacks wooden crates next to pile of plastic crates.</s>",
"The relation is entailment because the premise provides additional information that supports the hypothesis. The hypothesis states that the man is working hard and the premise describes the man in specific detail, stacking wooden crates next to a pile of plastic crates. This action of stacking crates is an indication that the man is indeed working hard.",




"sample 50: explain why the relation is entailment between hypothesis: two humans are having a meal. and premise: these two people are eating.</s>",
"The relation is entailment between hypothesis: \"two humans are having a meal\" and premise: \"these two people are eating.\" because the premise \"these two people are eating\" is a specific instance of the more general statement in the hypothesis \"two humans are having a meal.\" The premise provides additional information that the two individuals in question are indeed people and that they are eating, which confirms the hypothesis.",




"sample 51: explain why the relation is neutral between hypothesis: the woman is walking back to her car after shopping. and premise: a woman wearing a blue dress and carrying a purple umbrella is walking in parking lot.</s>",
"The relation is neutral between the hypothesis \"the woman is walking back to her car after shopping\" and the premise \"a woman wearing a blue dress and carrying a purple umbrella is walking in a parking lot\" because the premise provides some information about the woman's appearance and location, but does not confirm or deny the specific action of her returning to her car after shopping. The premise does not provide enough information to confirm or contradict the hypothesis.",




"sample 52: explain why the relation is neutral between hypothesis: a man wearing a coat stands on a hill. and premise: a man wearing a yellow coat stands at the peak of a mountain.</s>",
"The relation between the hypothesis \"a man wearing a coat stands on a hill\" and the premise \"a man wearing a yellow coat stands at the peak of a mountain\" is neutral. This is because the hypothesis provides information about the man wearing a coat and standing on a hill, while the premise provides information about the man wearing a yellow coat and standing at the peak of a mountain. Both the hypothesis and the premise contain information about a man wearing a coat and standing on a hill, but the premise also includes additional information about the man's coat being yellow and him standing at the peak of a mountain. The additional information in the premise doesn't contradict or entail the information in the hypothesis, it's just supplementary information.",




"sample 53: explain why the relation is entailment between hypothesis: a boy in an ironic shirt cries. and premise: a young baby boy crying while wearing a shirt that says \"i am the boss.\"</s>",
"The relation is entailment because the premise provides additional information that the boy is crying, and also that he is wearing an ironic shirt. The hypothesis states that the boy is crying and the premise confirms this by stating that the boy is crying, while also providing additional information about the shirt he is wearing. Therefore, the premise logically follows from the hypothesis.",




"sample 54: explain why the relation is neutral between hypothesis: three people play gin rummy. and premise: two men and a woman playing cards.</s>",
"The relation is neutral between the hypothesis \"three people play gin rummy\" and the premise \"two men and a woman playing cards\" because while the premise states that there are two men and a woman playing cards, it does not specify the game they are playing. Therefore, it is possible that they could be playing gin rummy, but it is not confirmed.",




"sample 55: explain why the relation is neutral between hypothesis: the girl's costume is too short. and premise: a dancer performing with her panties showing.</s>",
"The relation is neutral between the hypothesis \"the girl's costume is too short\" and the premise \"a dancer performing with her panties showing\" because the hypothesis states a specific condition about the girl's costume (i.e., that it is too short), while the premise describes a general situation (i.e., a dancer performing) without specifying anything about the costume. It is possible that the dancer in the premise is the girl mentioned in the hypothesis, but without more information it is not clear if that is the case. Therefore, the two statements do not directly support or contradict each other.",




"sample 56: explain why the relation is contradiction between hypothesis: two boys ride regular bicycles during a race. and premise: two boys with unicycles stand outside of a street shop.</s>",
"The relation is contradiction between the hypothesis \"two boys ride regular bicycles during a race\" and the premise \"two boys with unicycles stand outside of a street shop\" because the hypothesis states that the boys are riding regular bicycles, while the premise states that they are riding unicycles. These two statements cannot both be true at the same time, as a regular bicycle and a unicycle are two different types of vehicles and therefore are mutually exclusive. Therefore, the relation is contradiction.",




"sample 57: explain why the relation is entailment between hypothesis: the two women are riding a cart outside. and premise: two asia women are riding on a cart as a man pulls them through the street.</s>",
"The relation is entailment between the hypothesis \"the two women are riding a cart outside\" and the premise \"two asia women are riding on a cart as a man pulls them through the street\" because the premise provides specific details that support and expand upon the information in the hypothesis. The hypothesis states that the women are riding a cart, and the premise provides additional information that they are Asian women, and that they are being pulled through the street by a man. This additional information supports and adds to the information provided in the hypothesis, making the premise an entailment of the hypothesis.",




"sample 58: explain why the relation is contradiction between hypothesis: boys are singing in the public. and premise: two male and six female dancers perform on stage.</s>",
"The relation is contradiction between hypothesis: boys are singing in the public. and premise: two male and six female dancers perform on stage, because the hypothesis states that boys are singing, while the premise states that two males and six females are dancing. These two statements cannot both be true at the same time, therefore they contradict each other.",




"sample 59: explain why the relation is contradiction between hypothesis: the kids are frowning and premise: children smiling and waving at camera</s>",
"The relation is contradiction because the hypothesis states that the kids are frowning, which is the opposite of the premise that states the children are smiling and waving at the camera. The two statements are mutually exclusive and cannot both be true at the same time.",




"sample 60: explain why the relation is neutral between hypothesis: three men in black shirts are at a table. and premise: three men, one of them standing, are at a table in a dark room.</s>",
"The relation between the hypothesis \"three men in black shirts are at a table\" and the premise \"three men, one of them standing, are at a table in a dark room\" is neutral because the premise does not provide enough information to confirm or contradict the hypothesis. The premise states that there are three men at a table, but it does not specify their clothing or the lighting conditions, so it is not possible to determine whether they match the description in the hypothesis.",





"sample 61: explain why the relation is contradiction between hypothesis: a person is at a diner, ordering an omelette. and premise: a person on a horse jumps over a broken down airplane.</s>",
"The relation is a contradiction because the premise describes a person engaging in an activity that is not consistent with the setting described in the hypothesis. In the hypothesis, the person is at a diner ordering an omelette, which implies they are in an indoor, urban setting. However, in the premise, the person is on a horse jumping over a broken down airplane, which implies they are in a rural or wilderness setting, not a diner. These two settings are not compatible and therefore the hypothesis and premise contradict each other.",





"sample 62: explain why the relation is contradiction between hypothesis: a cart is sinking in the ocean and premise: a donkey pulled cart is carrying produce down a street.</s>",
"The relation is a contradiction between the hypothesis and the premise because the hypothesis states that a cart is sinking in the ocean, which implies that it is in water and unable to be used. However, the premise states that a donkey pulled cart is carrying produce down a street, which implies that the cart is on land and being used for transportation. These two statements cannot both be true at the same time and thus, the relation between the hypothesis and the premise is a contradiction.",




"sample 63: explain why the relation is entailment between hypothesis: a basketball game is being played. and premise: a group of basketball players jump in the air for the ball.</s>",
"The relation is entailment because the premise \"a group of basketball players jump in the air for the ball\" implies that a basketball game is being played. The action described in the premise, the players jumping for the ball, is a specific aspect of a basketball game and therefore the premise supports the hypothesis \"a basketball game is being played.\"",




"sample 64: explain why the relation is neutral between hypothesis: a person is wearing his church clothes as he lays on the ground. and premise: a person wearing a blue shirt, gray pants, and black shoes is laying on the ground next to a crutch.</s>",
"The relation is neutral because the premise does not specify that the person is wearing \"church clothes\" as the hypothesis states. The premise provides a description of the person's clothing, but it does not provide enough information to confirm or contradict the hypothesis that the person is wearing \"church clothes.\"",




"sample 65: explain why the relation is entailment between hypothesis: a large brown dog is standing oustide. and premise: a large brown dog is sticking his face in the sprinkler.</s>",
"The relation is entailment because the premise provides additional information about the large brown dog in the hypothesis, specifically that it is standing outside and sticking its face in a sprinkler, which is an action that a dog standing outside may do. The premise does not contradict the hypothesis, but rather adds to it.",




"sample 66: explain why the relation is neutral between hypothesis: women are standing on the sidewalk. and premise: two women conversing on a sidewalk.</s>",
"The relation is neutral between the hypothesis \"women are standing on the sidewalk\" and the premise \"two women conversing on a sidewalk\" because the hypothesis states that the women are standing, while the premise does not specify whether they are standing or sitting. Both statements are true, but the premise doesn't give additional information to the hypothesis.",




"sample 67: explain why the relation is entailment between hypothesis: a man is being carried. and premise: the long-haired man is balancing on a drum that is being carried by men in white shirts.</s>",
"The relation is entailment because the premise \"the long-haired man is balancing on a drum that is being carried by men in white shirts\" provides more information and context about the hypothesis \"a man is being carried.\" The additional information in the premise confirms that the man is indeed being carried by other men and adds detail to the image of the scene.",




"sample 68: explain why the relation is neutral between hypothesis: the man is very thirsty. and premise: man sitting by himself on a pier with a bottle of water.</s>",
"The relation is neutral because the premise provides information about a man sitting with a bottle of water, but it does not provide any information about his level of thirst. The hypothesis states that the man is very thirsty, but this information is not confirmed by the premise.",




"sample 69: explain why the relation is neutral between hypothesis: the 3 people are trying to figure out what the sign on the door means and are a little worried that they may be in the wrong room. and premise: two women and a man in a room that has a sign reading \"artisti\" on its door.</s>",
"The relation is neutral because while the hypothesis suggests that the three people are trying to figure out what the sign on the door means and are a little worried that they may be in the wrong room, the premise simply states that two women and a man are in a room that has a sign reading \"artisti\" on its door. The premise does not provide any information about the people's actions or emotions, so it cannot confirm or deny the hypothesis.",




"sample 70: explain why the relation is contradiction between hypothesis: man sets in cage and plays with rabbits. and premise: woman sets in cage and plays with rabbits.</s>",
"The relation is contradiction because the hypothesis states that a man is in the cage and playing with rabbits, while the premise states that a woman is in the cage and playing with rabbits. These statements cannot both be true at the same time, as they describe different genders of the person in the cage. This is a clear contradiction.",





"sample 71: explain why the relation is entailment between hypothesis: at least two people are in the pool. and premise: the little boy is in the swimming pool with an adult woman.</s>",
"The relation is entailment because the premise states that there is at least one adult woman and one little boy in the pool, which meets the minimum requirement stated in the hypothesis of \"at least two people\" being in the pool. The premise provides additional information about the specific individuals in the pool, but it does not contradict the statement in the hypothesis.",




"sample 72: explain why the relation is entailment between hypothesis: men and women are working in a neighborhood. and premise: a group of roughly 14 men and women works at a construction site in a residential area.</s>",
"The relation is entailment because the hypothesis states that \"men and women are working in a neighborhood\" and the premise states that \"a group of roughly 14 men and women works at a construction site in a residential area.\" The premise provides specific details that support the general statement made in the hypothesis, and therefore the premise entails the hypothesis. In other words, if the premise is true, the hypothesis is also true.",




"sample 73: explain why the relation is contradiction between hypothesis: the woman is cooking dinner. and premise: a woman is bending over a colorfully striped, metal couch on a sidewalk that has a bucket sitting atop it.</s>",
"The relation is contradiction because the hypothesis suggests the woman is engaged in the activity of cooking dinner, which typically takes place indoors and involves the use of a kitchen, whereas the premise describes the woman as being outdoors on a sidewalk, in proximity to a couch and a bucket. These two descriptions are not consistent with each other, and therefore the hypothesis and premise contradict each other.",




"sample 74: explain why the relation is neutral between hypothesis: two vehicles were parked next to a toyota. and premise: a black truck and a red car parked on the street in front of a grocery store painted red.</s>",
"The relation is neutral between the hypothesis \"two vehicles were parked next to a toyota\" and the premise \"a black truck and a red car parked on the street in front of a grocery store painted red\" because while the premise provides information about two specific vehicles (a black truck and a red car) being parked on the street, it does not specify the location of the vehicles in relation to one another. The hypothesis implies that there is a Toyota parked next to the two vehicles, but this information is not provided in the premise. Therefore, the premise does not confirm or deny the claim made in the hypothesis.",




"sample 75: explain why the relation is neutral between hypothesis: she is teaching her son to read. and premise: a mother is sitting and reading with her son.</s>",
"The relation is neutral between the hypothesis \"she is teaching her son to read\" and the premise \"a mother is sitting and reading with her son\" because the premise provides no information on whether the mother is actively teaching the son to read or if the son is reading independently with the help of the mother. The premise simply states that the mother is sitting and reading with her son, which could be interpreted as teaching, but it is not explicitly stated.",




"sample 76: explain why the relation is entailment between hypothesis: two people are by a tree. and premise: two kids stand under an area carved in a large tree.</s>",
"The relation is entailment between the hypothesis \"two people are by a tree\" and the premise \"two kids stand under an area carved in a large tree\" because the premise provides additional information about the location of the two people (i.e., that they are standing under an area carved in a large tree) and confirms that the hypothesis is true. The premise also provides information about the two people being kids, which is not mentioned in the hypothesis but it is entailed by it.",




"sample 77: explain why the relation is neutral between hypothesis: it is raining and premise: a young boy wearing a bright yellow raincoat, jean shorts and sandals squats down as he looks at an older boy and girl who are standing next to him and looking down some railroad tracks.</s>",
"The relation is neutral because the premise describes a specific scene with specific details, while the hypothesis simply states that it is raining. The premise does not confirm or deny the hypothesis that it is raining, it simply describes a scene that could be happening in the rain. While the boy is wearing a yellow raincoat, it doesn't confirm that it is raining, it could be worn for protection. Thus the premise does not provide any information that would contradict the hypothesis.",




"sample 78: explain why the relation is entailment between hypothesis: the people are going upstairs. and premise: several people walk up a staircase.</s>",
"The relation between the hypothesis \"the people are going upstairs\" and the premise \"several people walk up a staircase\" is entailment, because the premise provides additional information that confirms the hypothesis. The hypothesis states that the people are going upstairs, and the premise shows that they are physically walking up a staircase, which confirms that they are indeed going upstairs.",




"sample 79: explain why the relation is neutral between hypothesis: an older man drinks his juice as he waits for his daughter to get off work. and premise: an older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.</s>",
"The relation is neutral between the hypothesis and premise because while the hypothesis states that the older man is drinking his juice and waiting for his daughter to get off work, the premise only describes the older man sitting with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background. The premise does not provide any information about the older man's daughter or why he is waiting. Therefore, the hypothesis and premise are not mutually exclusive or mutually dependent and do not contradict or entail each other.",




"sample 80: explain why the relation is contradiction between hypothesis: the man is standing. and premise: a man sitting on a curb in a city park.</s>",
"The relation is a contradiction because the hypothesis states that the man is standing, while the premise states that the man is sitting on a curb. These two statements cannot both be true at the same time, as standing and sitting are mutually exclusive actions. Therefore, the hypothesis and the premise are in direct opposition to each other.",




"sample 81: explain why the relation is entailment between hypothesis: a man is taking a run on the track. and premise: a man running on a track.</s>",
"The relation between the hypothesis \"a man is taking a run on the track\" and the premise \"a man running on a track\" is entailment because the premise provides additional information that confirms the hypothesis. The hypothesis states that a man is taking a run, which implies that he is moving, and the premise confirms that the man is indeed running, which is a specific type of movement. Therefore, the premise provides evidence that supports the hypothesis, making them consistent with each other.",




"sample 82: explain why the relation is neutral between hypothesis: a person is resting. and premise: a man crouching down on some rocks in the ocean.</s>",
"The relation is neutral because the premise describes a specific action (crouching on rocks) that is not necessarily indicative of resting. The hypothesis states that the person is resting, but the premise does not provide any information to confirm or deny that statement.",




"sample 83: explain why the relation is entailment between hypothesis: a woman stands at a podium. and premise: a middle-aged woman stands at a green podium with the name \"sva theatre\" written on it.</s>",
"The relation is entailment between the hypothesis \"a woman stands at a podium\" and the premise \"a middle-aged woman stands at a green podium with the name \"sva theatre\" written on it\" because the premise provides additional information that supports the hypothesis. The hypothesis states that a woman is standing at a podium, and the premise states that a middle-aged woman is standing at a green podium with a specific name, \"sva theatre\" written on it, which confirms the hypothesis that a woman is standing at a podium. Therefore, the premise entails the hypothesis.",




"sample 84: explain why the relation is contradiction between hypothesis: a man wearing a cowboy hat swings a baseball bat. and premise: a man wearing only a cowboy hat, cowboy boots, and underwear talks to a woman on a city street.</s>",
"The relation is contradiction between the hypothesis \"a man wearing a cowboy hat swings a baseball bat\" and the premise \"a man wearing only a cowboy hat, cowboy boots, and underwear talks to a woman on a city street\" because the premise states that the man is only wearing a cowboy hat, cowboy boots, and underwear and is talking to a woman on a city street, while the hypothesis states that the man is wearing a cowboy hat and swinging a baseball bat, which are two completely different and mutually exclusive actions and scenarios. Therefore, the hypothesis and the premise are contradictory.",




"sample 85: explain why the relation is contradiction between hypothesis: the man is lying in bed. and premise: a man and woman dressed in old fashion clothing sit outside at a table.</s>",
"The relation is a contradiction because the hypothesis states that the man is lying in bed, which is an indoor and sedentary activity, while the premise describes the man and woman sitting outside at a table, which is an outdoor and active activity. These two descriptions are mutually exclusive and cannot both be true at the same time, hence the contradiction.",




"sample 86: explain why the relation is entailment between hypothesis: store has people standing around it's exterior. and premise: asian people are standing outside a store named concepts.</s>",
"The relation between the hypothesis \"store has people standing around it's exterior\" and the premise \"asian people are standing outside a store named concepts\" is entailment because the premise provides evidence that supports the hypothesis. The premise specifically states that there are people standing outside a store, and the store is named \"concepts\", which confirms that there is a store with people standing around its exterior. Therefore, the premise entails the hypothesis, meaning that the hypothesis is logically supported by the premise.",





"sample 87: explain why the relation is contradiction between hypothesis: there are people running and premise: couple walking passed a clock tower.</s>",
"The relation is a contradiction between the hypothesis \"there are people running\" and the premise \"couple walking passed a clock tower\" because the hypothesis states that people are running, while the premise states that people are walking. These two statements are incompatible and cannot both be true at the same time. The fact that people are running implies that they are moving quickly and with purpose, while the fact that people are walking implies a slower and more leisurely pace. The contradiction is that the two statements give different information about the activity of the people.",




"sample 88: explain why the relation is contradiction between hypothesis: the men are watching a movie in public theatres in different states and premise: two men handle a large telescope while one is looking through.</s>",
"The relation is contradiction because the hypothesis states that the men are watching a movie in public theatres in different states, while the premise states that two men are handling a large telescope and one is looking through it. These two statements are mutually exclusive, as it is unlikely that the men would be able to watch a movie in public theatres in different states while also handling a telescope at the same time. Therefore, the hypothesis and premise are contradictory.",




"sample 89: explain why the relation is entailment between hypothesis: people are playing. and premise: a group of native adults and children are sitting in a circle playing a game.</s>",
"The relation between the hypothesis \"people are playing\" and the premise \"a group of native adults and children are sitting in a circle playing a game\" is entailment because the premise provides specific information that supports the general idea presented in the hypothesis. The premise describes a specific instance of people playing, which is what the hypothesis states is happening. The premise provides additional details, such as that the people are native and that they are playing a game, but these details do not contradict or negate the general idea that people are playing, as stated in the hypothesis. Therefore, the premise entails the hypothesis.",




"sample 90: explain why the relation is neutral between hypothesis: the men are related and premise: a man sitting with an elderly homeless person in a park.</s>",
"The relation is neutral because the premise does not provide any information about the relationship between the two men. It only states that they are sitting together in a park. The hypothesis, on the other hand, states that they are related, but it is not specified how they are related, and there is no information from the premise that can confirm or deny this claim. Therefore, it is not possible to determine if the hypothesis is true or false based on the premise.",




"sample 91: explain why the relation is neutral between hypothesis: a man is competing on a bike. and premise: a male doing a trick in the air on his bike.</s>",
"The relation is neutral between the hypothesis \"a man is competing on a bike\" and the premise \"a male doing a trick in the air on his bike\" because the premise does not indicate whether the man is competing or not, it only describes an action of a man on a bike. The premise does not provide any information about a competition or a race, so it does not confirm or contradict the hypothesis.",




"sample 92: explain why the relation is neutral between hypothesis: some artists are painting a wall. and premise: four young women are painting in countries of a map that is painted on a wall.</s>",
"The relation is neutral between the hypothesis \"some artists are painting a wall\" and the premise \"four young women are painting in countries of a map that is painted on a wall\" because the premise describes a specific situation, but the hypothesis is more general and does not specify that the artists are women or that they are painting a map on a wall. The hypothesis is compatible with the premise, but the premise does not provide any additional information or context to confirm or deny the hypothesis.",




"sample 93: explain why the relation is contradiction between hypothesis: a woman takes a bath. and premise: three people serving food outside somewhere.</s>",
"The relation is contradiction between the hypothesis \"a woman takes a bath\" and the premise \"three people serving food outside somewhere\" because the two statements describe two completely different and unrelated actions that cannot coexist at the same time. The hypothesis describes an individual doing a solitary activity in a private setting, while the premise describes a group of individuals performing an interactive activity in a public setting. The two statements are not compatible with each other and therefore contradict each other.",




"sample 94: explain why the relation is neutral between hypothesis: the woman is trying to make a living. and premise: an asian woman is selling colorful fabrics.</s>",
"The relation is neutral between the hypothesis \"the woman is trying to make a living\" and the premise \"an asian woman is selling colorful fabrics\" because while the premise describes an action that may be associated with someone trying to make a living, it does not directly confirm or refute the hypothesis. The premise simply provides a visual image of the woman selling fabrics, without any additional context or information about her motivations or circumstances.",





"sample 95: explain why the relation is contradiction between hypothesis: the boy skates down the sidewalk. and premise: a boy is jumping on skateboard in the middle of a red bridge.</s>",
"The relation is a contradiction between the hypothesis \"the boy skates down the sidewalk\" and the premise \"a boy is jumping on a skateboard in the middle of a red bridge\" because the hypothesis suggests that the boy is moving on the sidewalk, while the premise suggests that the boy is on a skateboard in the middle of a red bridge. These two statements cannot both be true at the same time, as the sidewalk and red bridge are different locations and the boy cannot be in both places simultaneously. Additionally, the action described in the hypothesis \"skating down\" is different from the action described in the premise \"jumping on\". Therefore, the two statements contradict each other.",
]

In [27]:
# import wandb
# training_args.run_name=""
training_args.logging_steps=3
training_args.save_steps=10
training_args.evaluation_strategy="epoch"
training_args.num_train_epochs=15
training_args.do_eval=True

In [28]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./esnli_output_t5_base/020123_203123,
logging_first_step=False,
logging_steps=3,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_paramete

In [29]:
training_args.per_device_eval_batch_size=8
training_args.per_device_train_batch_size=8
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./esnli_output_t5_base/020123_203123,
logging_first_step=False,
logging_steps=3,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.LINEAR,
max_grad_norm=1.0,
max_steps=-1,
metric_for_best_model=None,
mp_paramete

In [30]:
class SequenceCollator:
    def __init__(self, model, pad_token):
        self.model = model
        self.pad_token_mapping = {
            "labels": -100,
            "attention_mask": 0,
            "decoder_attention_mask": 0,
            "input_ids": pad_token,
        }

        self.columns = [
            "input_ids",
            "attention_mask",
            "labels",
            "decoder_attention_mask",
        ]

    def __call__(self, examples: List[Dict[str, InputDataClass]]) -> Dict[str, torch.Tensor]:
        # re-format inputs for training
        batch = {}
        for key in examples[0].keys():
            if key in self.columns:
                tmp_list = []
                for item in examples:
                    tmp_list.append(item[key])

                # pad lists to max length
                if isinstance(tmp_list[0], list):
                    max_length = max(map(len, tmp_list))
                    tmp_list = [
                        el + [self.pad_token_mapping[key]] * (max_length - len(el))
                        for el in tmp_list
                    ]

                batch[key] = torch.tensor(tmp_list, dtype=torch.long)
        return batch

In [31]:
# os.environ["WANDB_DISABLED"] = "True"
if data_args.generations_filepath is None:
    callbacks = [TensorBoardCallback()]
    if data_args.early_stopping_patience > 0:
        callbacks.append(EarlyStoppingCallback(early_stopping_patience=data_args.early_stopping_patience))
        training_args.load_best_model_at_end = True
    else:
        training_args.load_best_model_at_end = False  # use the last model state
    training_args.metric_for_best_model = 'eval_loss'
    training_args.greater_is_better = False
    if training_args.eval_steps is None:
        training_args.evaluation_strategy = EvaluationStrategy.EPOCH
    else:
        training_args.evaluation_strategy = EvaluationStrategy.STEPS

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=our_data_splits['train'],
        eval_dataset=our_data_splits['dev'],
        data_collator=SequenceCollator(
            model=model_class, pad_token=tokenizer.pad_token_id
        ),
        callbacks=callbacks,
    )

# Training. Don't train if it is use_gpt3
if training_args.do_train and not model_args.use_gpt3:
    start_time = time.time()
    trainer.train()
    train_time = time.time() - start_time
    model = trainer.model
    #wandb.finish()
else:
    start_time = time.time()
    train_time = time.time() - start_time

You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
WandbCallback
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: explanation_2, hypothesis, explanation_1, our_explanation, explanation_3, premise, question_encoding.
***** Running training *****
  Num examples = 72
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 135
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: dengdenghuang (cuhk_lavilab). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


Saving model checkpoint to ./esnli_output_t5_base/020123_203123/checkpoint-10
Configuration saved in ./esnli_output_t5_base/020123_203123/checkpoint-10/config.json
Model weights saved in ./esnli_output_t5_base/020123_203123/checkpoint-10/pytorch_model.bin
Saving model checkpoint to ./esnli_output_t5_base/020123_203123/checkpoint-20
Configuration saved in ./esnli_output_t5_base/020123_203123/checkpoint-20/config.json
Model weights saved in ./esnli_output_t5_base/020123_203123/checkpoint-20/pytorch_model.bin
Saving model checkpoint to ./esnli_output_t5_base/020123_203123/checkpoint-30
Configuration saved in ./esnli_output_t5_base/020123_203123/checkpoint-30/config.json
Model weights saved in ./esnli_output_t5_base/020123_203123/checkpoint-30/pytorch_model.bin
Saving model checkpoint to ./esnli_output_t5_base/020123_203123/checkpoint-40
Configuration saved in ./esnli_output_t5_base/020123_203123/checkpoint-40/config.json
Model weights saved in ./esnli_output_t5_base/020123_203123/checkpoi

# Rationale Discriminator

## Construction of generated rationale and accepted rationale pairs


In [32]:
# dict_keys(['premise', 'hypothesis', 'label',
#            'explanation_1', 'explanation_2', 'explanation_3', 
#            'our_explanation', 'common_expl_list'])

In [33]:
from tqdm import tqdm
# fse_csqa_dev_data_dict
# fse_csqa_train_data_dict
rationale_pair_dev_data = []
cc=0
good_model = model

for da in tqdm(our_data_splits['dev'], total=len(our_data_splits['dev'])):
    print("*******")
    print("hypothesis: {}".format(da['hypothesis']))
    print("premise: {}".format(da['premise']))
    print("answer: {}".format(da['label']))
    
    #print("our_explanation: {}".format(da['our_explanation']))
    #id_ = da['id']
#     if id_ in fse_csqa_dev_data_dict.keys():
#         common_expl_list = fse_csqa_dev_data_dict[id_]['common_expl_list']
#     else:
#         common_expl_list = fse_csqa_train_data_dict[id_]['common_expl_list']
#     da["common_expl_list"] = common_expl_list
    print("common expl list: {}".format(da["common_expl_list"]))
    inp_ids = torch.tensor(da["input_ids"], device=model.device).reshape(1, -1)
    out = good_model.generate(
                    inp_ids,
                    max_length=100,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
    skip_special_tokens = False if "infilling" in data_args.io_format else True
    words = tokenizer.decode(out[0].tolist(), skip_special_tokens=skip_special_tokens)
    print("generated explanation: {}".format(words))
    da["generated_explanation"] = words
    print("########")
    rationale_pair_dev_data.append(da)
    cc += 1
#     if cc > 200:
#         break
    #pdb.set_trace()

  0%|                                                                                                                                                                               | 0/640 [00:00<?, ?it/s]

*******
hypothesis: There is a black dog.
premise: a black dog running really fast with a blond woman running after him in the background.
answer: 0
common expl list: ['If the dog is black, then it is black.', "If there's a black dog running really fast, then, yes, there is a black dog."]


  0%|▎                                                                                                                                                                      | 1/640 [00:01<14:09,  1.33s/it]

generated explanation: <pad> <extra_id_0> The relation is entailment because the premise describes a blond woman running after the dog, which is a black dog, which is a fast dog, which is a black dog running really fast, which is a blond dog, which is a black dog, which is a blond dog, which is a blond dog, which is a dog, which is a blond woman, which is
########
*******
hypothesis: people at home
premise: a crowd of people in a subway
answer: 2
common expl list: ['you cannot be at home in a subway. Can you?']


  0%|▌                                                                                                                                                                      | 2/640 [00:03<16:58,  1.60s/it]

...

generated explanation: <pad> <extra_id_0> The relation between the hypothesis "the man is walking into a room" and the premise "a man is standing in the doorway of a building" is neutral because the premise does not confirm or deny the hypothesis. The premise does not support or deny the hypothesis.<extra_id_1> </s>
########
*******
hypothesis: The log is for firewood.
premise: In a snowy residential environment, a man with a brown jacket and gray mustache and beard places a hand on a felled log while another man in a blue shirt is blurry on the right side of the photo.
answer: 1
common expl list: ['The log could be for firewood, but it could also be for a number of other things.', 'While firewood is typically cut logs, just because a log is present does not mean it is for firewood.']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 639/640 [14:29<00:00,  1.01it/s]

generated explanation: <pad> <extra_id_0> The relation is neutral because the premise provides no information about the log and the hand on the log, which is the main source of firewood, and the premise does not provide any information about the log. Therefore, the hypothesis "the log is for firewood" is not related to the premise.<extra_id_1> </s>
########
*******
hypothesis: A child uses their imagination.
premise: A child making a sand castle on the beach.
answer: 1
common expl list: ['A child could be using their imagination while building a sand castle, but they could also be building a sand castle without using their imagination.']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 640/640 [14:31<00:00,  1.36s/it]

generated explanation: <pad> <extra_id_0> The relation is neutral between the hypothesis "a child uses their imagination" and the premise "a child making a sand castle on the beach" because the premise does not provide enough information to confirm or deny the hypothesis. The premise provides additional information about the child making a sand castle on the beach. The child is a child, and the premise provides no information about the child's imagination. The premise does not provide enough information
########


In [34]:
import json
rationale_pair_save_path = os.path.join("./results", "72shots_esnli_t5_base_chatgpt_rationales_generator_test_rationale_pair.json")
with open(rationale_pair_save_path, 'w') as f:
    json.dump(rationale_pair_dev_data, f)